In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
# %watermark -p numpy,sklearn,pandas
# %watermark -p ipywidgets,cv2,PIL,matplotlib,plotly,netron
# %watermark -p torch,torchvision,torchaudio
# %watermark -p tensorflow,tensorboard,tflite
# %watermark -p onnx,tf2onnx,onnxruntime,tensorrt,tvm
# %matplotlib inline
# %config InlineBackend.figure_format='retina'
# %config IPCompleter.use_jedi = False

# from IPython.display import display, Markdown, HTML, IFrame, Image, Javascript
# from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
# display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

import sys, os, io, logging, time, random, math
import json, base64, requests, shutil
import argparse, shlex, signal, traceback
import numpy as np

argparse.ArgumentParser.exit = lambda *arg, **kwargs: _IGNORE_

def _IMPORT(x, debug=False):
    try:
        x = x.strip()
        if x[0] == '/' or x[1] == '/':
            with open(x) as fr:
                x = fr.read()
        elif 'github' in x or 'gitee' in x:
            if x.startswith('import '):
                x = x[7:]
            if x.startswith('https://'):
                x = x[8:]
            if not x.endswith('.py'):
                x = x + '.py'
            x = x.replace('blob/main/', '').replace('blob/master/', '')
            if x.startswith('raw.githubusercontent.com'):
                x = 'https://' + x
                x = requests.get(x)
                if x.status_code == 200:
                    x = x.text
            elif x.startswith('github.com'):
                x = x.replace('github.com', 'raw.githubusercontent.com')
                mod = x.split('/')
                for s in ['/main/', '/master/']:
                    x = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[-3:])
                    x = requests.get(x)
                    if x.status_code == 200:
                        x = x.text
                        break
            elif x.startswith('gitee.com'):
                mod = x.split('/')
                for s in ['/raw/main/', '/raw/master/']:
                    x = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:])
                    if debug:
                        print(x)
                    x = requests.get(x)
                    if x.status_code == 200:
                        x = x.text
                        break
        if debug:
            return x
        else:
            exec(x, globals())
    except Exception as err:
        # sys.stderr.write(f'request {x} : {err}')
        pass

def _DIR(x, dumps=True, ret=True):
    attrs = sorted([y for y in dir(x) if not y.startswith('_')])
    result = '%s: %s' % (str(type(x))[8:-2], json.dumps(attrs) if dumps else attrs)
    if ret:
        return result
    print(result)
    

In [2]:
###
### Display ###
###

_IMPORT('import pandas as pd')
_IMPORT('import cv2')
_IMPORT('from PIL import Image')
_IMPORT('import matplotlib.pyplot as plt')
_IMPORT('import plotly')
_IMPORT('import plotly.graph_objects as go')
_IMPORT('import ipywidgets as widgets')
_IMPORT('from ipywidgets import interact, interactive, fixed, interact_manual')

# plotly.offline.init_notebook_mode(connected=False)

plt.rcParams['figure.figsize'] = (12.0, 8.0)

def show_image(imgsrc, width=None, height=None):
    if isinstance(imgsrc, np.ndarray):
        img = imgsrc
        if width or height:
            if width and height:
                size = (width, height)
            else:
                rate = img.shape[1] / img.shape[0]
                if width:
                    size = (width, int(width/rate))
                else:
                    size = (int(height*rate), height)
            img = cv2.resize(img, size)
            plt.figure(figsize=(3*int(size[0]/80+1), 3*int(size[1]/80+1)), dpi=80)
        plt.axis('off')
        if len(img.shape) > 2:
            plt.imshow(img);
        else:
            plt.imshow(img, cmap='gray');
        return

    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if imgsrc.startswith('http'):
        data_url = imgsrc
    else:
        if len(imgsrc) > 2048:
            data_url = 'data:image/jpg;base64,' + imgsrc
        else:
            img = open(imgsrc, 'rb').read()
            data_url = 'data:image/jpg;base64,' + base64.b64encode(img).decode()
    return HTML('<center><img %s %s src="%s"/></center>' % (W, H, data_url))

def nbeasy_widget_display(images, img_wid=None):
    if isinstance(images, np.ndarray):
        images = {'_': images}
    elif isinstance(images, tuple) or isinstance(images, list):
        images = {f'_{i}': img for i, img in enumerate(images)}
    C = len(images)
    if C == 0:
        return None
    show_ncol, show_nrow = 1, 1
    if C > 1:
        if img_wid:
            show_ncol = 2 if int(img_wid.width) <= 720 else 1
        for i in range(C % show_ncol):
            images[f'placehold-{i}'] = images[list(images.keys())[-1]].copy()
        show_nrow = len(images) // show_ncol
        row_images = []
        col_images = []
        for key, img in images.items():
            if not key.startswith('_'):
                cv2.putText(img, key, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 9,2), 1)
            col_images.append(img)
            if len(col_images) == show_ncol:
                row_images.append(np.hstack(col_images))
                col_images = []

        display_image = np.vstack(row_images)
    else:
        display_image = images.popitem()[1]
    
    imgbytes = io.BytesIO(cv2.imencode('.png', display_image)[1]).getvalue()
    if img_wid:
        img_wid.layout.width = f'{display_image.shape[1]}px'
        img_wid.layout.height = f'{display_image.shape[0]}px'
        img_wid.value = imgbytes
    else:
        return imgbytes

# https://www.webucator.com/article/python-color-constants-module/
class COLORS(object):
    # BGR
    BLUE       = (255 , 0   , 0)
    GREEN      = (0   , 255 , 0)
    RED        = (0   , 0   , 255)
    BLACK      = (0   , 0   , 0)
    YELLOW     = (0   , 255 , 255)
    WHITE      = (255 , 255 , 255)
    CYAN       = (255 , 255 , 0)
    MAGENTA    = (255 , 0   , 242)
    GOLDEN     = (32  , 218 , 165)
    LIGHTBLUE  = (255 , 9   , 2)
    PURPLE     = (128 , 0   , 128)
    CHOCOLATE  = (30  , 105 , 210)
    PINK       = (147 , 20  , 255)
    ORANGE     = (0   , 69  , 255)
    GRAY       = (125 , 125 , 125)
    DARKGRAY   = (50  , 50  , 50)
    DARKGREEN  = (0   , 100 , 0)
    OLIVE      = (0   , 128 , 0)
    SILVER     = (192 , 192 , 192)

In [3]:
import json, functools, datetime

class __JsonEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (datetime.datetime, datetime.timedelta)):
            return '{}'.format(obj)
        elif isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return json.JSONEncoder.default(self, obj)

json.dumps = functools.partial(json.dumps, cls=__JsonEncoder)

In [4]:
import skimage.filters as filters
import threading
import shutil
import glob
import pickle
import collections
from enum import IntEnum

_IMPORT('gitee.com/qrsforever/nb_easy/easy_widget')

g_ctx = None

In [5]:
_IMPORT('./easy_widget.py')

## Generate Grid Aurco

In [ ]:
# A4_width_mm, A4_height_mm, printer_dpi = 210, 297, 600
printer_dpi = 600
printer_dpmm = printer_dpi / 25.4
marker_size_mm, gap_size_mm = 25, 3
aurco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_250)

landscape = True # portrait

if landscape:
    A4_width_mm, A4_height_mm, printer_dpi = 280, 210, 600
else:
    A4_width_mm, A4_height_mm, printer_dpi = 210, 280, 600
    
markersX = (A4_width_mm - gap_size_mm) // (marker_size_mm + gap_size_mm)
markersY = (A4_height_mm - gap_size_mm) // (marker_size_mm + gap_size_mm)

aruco_gridboard = cv2.aruco.GridBoard_create(markersX, markersY, marker_size_mm, gap_size_mm, aurco_dict)

width = markersX * marker_size_mm  + (markersX - 1) * gap_size_mm
height =  markersY * marker_size_mm + (markersY - 1) * gap_size_mm

cv2.imwrite("aruco_gridboard.png", aruco_gridboard.draw((int(printer_dpmm * width), int(printer_dpmm * height))));

## Monocular: Single Camera Calibration

In [6]:
class CalibrateState(IntEnum):
    COLLECT = 1
    READIMAGE = 2
    CALIBRATE = 3
    COMPLETED = 4

def stop_camera_calibrate(ctx, w_btn_source=None):
    ctx.logger('stop_camera_calibrate()')
    if hasattr(ctx, 'camera') and ctx.camera:
        ctx.camera_is_running = False
        for i in range(3):
            if ctx.camera is None:
                break
            time.sleep(1)
        if ctx.camera:
            ctx.camera.release()

def save_calibration_intrinsics(camera_matrix, dist_coeffs, out_dir):
    cv_file = cv2.FileStorage(f'{out_dir}/intrinsics.yaml', cv2.FILE_STORAGE_WRITE)
    cv_file.write('camera_matrix', camera_matrix)
    cv_file.write('dist_coeffs', dist_coeffs)
    cv_file.release()

def load_calibration_intrinsics(out_dir):
    yaml_file = f'{out_dir}/intrinsics.yaml'
    assert os.path.exists(yaml_file), 'Not found intrinsics.yaml'
    cv_file = cv2.FileStorage(yaml_file, cv2.FILE_STORAGE_READ)
    camera_matrix = cv_file.getNode('camera_matrix').mat()
    dist_coeffs = cv_file.getNode('dist_coeffs').mat()
    return camera_matrix, dist_coeffs
 
def start_calibrate_camera_with_good_flags(ctx, calibrate, img_size):
    flags = 0
    K = np.array([[max(img_size), 0, img_size[1]/2],[0, max(img_size), img_size[0]/2], [0, 0, 1]])
    D = np.zeros((5, 1))
    
    retval, mat, dist, rvecs, tvecs, \
    std_intrinsics, std_extrinsics, errors = calibrate(imageSize=img_size, cameraMatrix=K, distCoeffs=D, flags=flags)

    ctx.logger(json.dumps({
        're-project-error': retval, 
        'mat': str(mat.round(3).tolist()),
        'dist': str(dist.round(3).tolist())}, indent=4))

    aspect_ratio = mat[0][0] / mat[1][1]
    if 1.0 - min(aspect_ratio, 1.0/aspect_ratio) < 0.01:
        flags += cv2.CALIB_FIX_ASPECT_RATIO
        retval, mat, dist, rvecs, tvecs, \
        std_intrinsics, std_extrinsics, errors = calibrate(imageSize=img_size, cameraMatrix=K, distCoeffs=D, flags=flags)
        ctx.logger(json.dumps({
            'aspect_ratio': aspect_ratio,
            're-project-error': retval, 
            'mat': str(mat.round(3).tolist()),
            'dist': str(dist.round(3).tolist())}, indent=4))

    center_point_reldiff = max(abs(np.array(mat[0, 2], mat[1][2]) - np.array(img_size)/2) / np.array(img_size))
    if center_point_reldiff < 0.05:
        flags += cv2.CALIB_FIX_PRINCIPAL_POINT
        retval, mat, dist, rvecs, tvecs, \
        std_intrinsics, std_extrinsics, errors = calibrate(imageSize=img_size, cameraMatrix=K, distCoeffs=D, flags=flags)
        ctx.logger(json.dumps({
            'center_point_reldiff': center_point_reldiff,
            're-project-error': retval, 
            'mat': str(mat.round(3).tolist()),
            'dist': str(dist.round(3).tolist())}, indent=4))

    error_threshold = 1.25 * retval
    camera_matrix = mat
    dist_coeffs = dist
    error = retval

    ignore_flags = {
        'ignore_tangential_distortion': cv2.CALIB_ZERO_TANGENT_DIST,
        'ignore_k3': cv2.CALIB_FIX_K3,
        'ignore_k2': cv2.CALIB_FIX_K2,
        'ignore_k1': cv2.CALIB_FIX_K1
    }

    for k, v in ignore_flags.items():
        flags += v
        retval, mat, dist, rvecs, tvecs, \
        std_intrinsics, std_extrinsics, errors = calibrate(imageSize=img_size, cameraMatrix=K, distCoeffs=D, flags=flags)
        ctx.logger(json.dumps({
            'ignore_type': k,
            're-project-error': retval, 
            'mat': str(mat.round(3).tolist()),
            'dist': str(dist.round(3).tolist())}, indent=4))
        if retval > error_threshold:
            break
        camera_matrix = mat
        dist_coeffs = dist
        error = retval
    
    return error, camera_matrix, dist_coeffs

def is_rotation_matrix(R):
    shouldBeIdentity = np.dot(np.transpose(R), R)
    return np.linalg.norm(np.identity(3, dtype=R.dtype) - shouldBeIdentity) < 1e-6

def rotation_matrix2euler_angles(R):
    assert(is_rotation_matrix(R))
    # pitch = np.arctan2(R[2, 1], R[2, 2])
    # raw = np.arctan2(-R[2, 0], np.hypot(R[2, 1], R[2, 2]))
    # roll = np.arctan2(R[1, 0], R[0, 0])
    sy = math.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0])
    singular = sy < 1e-6
    if not singular:
        pitch = math.atan2(R[2, 1], R[2, 2])
        yaw = math.atan2(-R[2, 0], sy)
        roll = math.atan2(R[1, 0], R[0, 0])
    else:
        pitch = math.atan2(-R[1, 2], R[1, 1])
        yaw = math.atan2(-R[2, 0], sy)
        roll = 0
    return np.array([pitch, yaw, roll])

def euler_angles2rotation_matrix(angles):
    # cos, sin = np.cos(angles), np.sin(angles)
    # X = np.array([
    #     [1,      0,       0],
    #     [0, cos[0], -sin[0]],
    #     [0, sin[0], cos[0]]])
    # Y = np.array([
    #     [cos[1],  0, sin[1]],
    #     [0,       1,      0],
    #     [-sin[0], 0, cos[0]]])
    # Z = np.array([
    #     [cos[2], -sin[2], 0],
    #     [sin[2],  cos[2], 0],
    #     [0,            0, 1]])
    # return Z @ Y @ X
    
    # Rotation on x-axis (pitch)
    X = np.array([
        [1, 0,                  0],
        [0, math.cos(angles[0]), -math.sin(angles[0])],
        [0, math.sin(angles[0]), math.cos(angles[0])]
    ])

    # Rotation on y-axis (raw)
    Y = np.array([
        [math.cos(angles[1]),  0, math.sin(angles[1])],
        [0,                    1, 0],
        [-math.sin(angles[1]), 0, math.cos(angles[1])]
    ])

    # Rotation on z-axis (roll)
    Z = np.array([
        [math.cos(angles[2]), -math.sin(angles[2]), 0],
        [math.sin(angles[2]), math.cos(angles[2]),  0],
        [0,                  0,                   1]
    ])
    return np.dot(Z, np.dot(Y, X))

def pixel_to_world(intrinsics_matrix, extrinsics_matrix, x, y):
    pseudo_inv_extrinsics = np.linalg.pinv(extrinsics_matrix)
    intrinsics_inv = np.linalg.inv(intrinsics_matrix)
    pixels_matrix = np.array((x, y, 1))
    ans = np.matmul(intrinsics_inv, pixels_matrix)
    ans = np.matmul(pseudo_inv_extrinsics, ans)
    ans /= ans[-1] 
    return ans

def measure_by_corners(camera_matrix, extrinsics_matrix, points):
    p1 = pixel_to_world(camera_matrix, extrinsics_matrix, points[0][0], points[0][1])
    p2 = pixel_to_world(camera_matrix, extrinsics_matrix, points[1][0], points[1][1])
    p3 = pixel_to_world(camera_matrix, extrinsics_matrix, points[2][0], points[2][1])
    W, H = p1 - p2, p2 - p3
    width = np.sqrt(W[0]**2 + W[1]**2)
    height = np.sqrt(H[0]**2 + H[1]**2)   
    return width, height

def single_aruco_estimate_pose(ctx, image, aruco_dict, camera_matrix, dist_coeffs, cross_line_p, marker_length=-1):
    marker_corners, marker_ids, _ = cv2.aruco.detectMarkers(image, aruco_dict)
    if marker_ids is None or len(marker_ids) > 5:
        return {'det_marker_image': image}
    det_marker_image = image
    measure_image = image.copy()
    tolerance_low, tolerance_high = cross_line_p - 3, cross_line_p + 3
    if np.all(marker_ids is not None):
        for (marker_corner, marker_id) in zip(marker_corners, marker_ids):
            # corner: [[[306. 237.]
            #          [440. 218.]
            #          [477. 359.]
            #          [344. 389.]]]
            if marker_length < 0:
                marker_length = 10.0 * (marker_id[0] - 60) # unit: mm
                if marker_length < 0:
                    continue
                    
            # another simple triangle method, as comparison with other methods
            pixel_cm_ratio = cv2.arcLength(marker_corner[0], True) / (4 * marker_length)
            min_rect = cv2.minAreaRect(marker_corner[0])
            (c_x, c_y), (m_w, m_h), angle = min_rect
            box = np.int0(cv2.boxPoints(min_rect))
            
            tri_width = m_w / pixel_cm_ratio
            tri_height = m_h / pixel_cm_ratio
            m_x, m_y, m_w, m_h, c_x, c_y = int(c_x - 0.5 * m_w), int(c_y - 0.5 * m_h), int(m_w), int(m_h), int(c_x), int(c_y)
            
            det_marker_image[m_y: m_y + m_h, m_x: m_x + m_w] = 127
            cv2.circle(det_marker_image, (int(c_x), int(c_y)), 2, COLORS.OLIVE, -1)
            cv2.polylines(det_marker_image, [box], True, COLORS.DARKGRAY, 2)

            cv2.aruco.drawDetectedMarkers(det_marker_image, marker_corners, marker_ids, borderColor=COLORS.CYAN)
            rvecs, tvecs, marker_points = cv2.aruco.estimatePoseSingleMarkers(marker_corner, marker_length, camera_matrix, dist_coeffs)

            # calculate the height/width (2D to 3D)
            (tl, tr, br, bl) = [(int(x[0]), int(x[1])) for x in marker_corner.reshape((4, 2))]

            # X (red color), Y (green color), Z (blue color)
            cv2.drawFrameAxes(det_marker_image, camera_matrix, dist_coeffs, rvecs, tvecs, int(.8 * marker_length))  # Draw Axis
            
            bx1, by1, bw, bh = cv2.boundingRect(marker_corner[0])
            if tolerance_low <= (by1 + bh) < tolerance_high:
                cv2.rectangle(det_marker_image, (bx1, by1), (bx1 + bw, by1 + bh), COLORS.GREEN, 3)
                
            # rvecs: [[[ 0.18987504  2.79373027 -0.9026566 ]]]
            # tvecs: [[[ 8.25180397  2.28427923 39.30899197]]]
            # rotation_matrix: [[-0.97191346,  0.18216257,  0.14900011],
            #                   [ 0.06054155,  0.80535491, -0.5896933 ],
            #                   [-0.22741802, -0.56411016, -0.79376368]]
            rotation_matrix, _ = cv2.Rodrigues(rvecs[0])
            extrinsics_matrix = np.concatenate([rotation_matrix, tvecs[0].T], axis=1) 
            width, height = measure_by_corners(camera_matrix, extrinsics_matrix, marker_corner[0])
            distance = np.linalg.norm(tvecs[0])
            # ctx.logger('width: {}, height: {}, distance: {}'.format(width, height, distance))
            # angles = cv2.decomposeProjectionMatrix(extrinsics_matrix)[6].flatten()
            angles = rotation_matrix2euler_angles(rotation_matrix)
            pitch, yaw, roll = [math.degrees(angle) for angle in angles]
            cv2.putText(det_marker_image,
                        'size {} x {} D:{}'.format(round(width, 1), round(height, 1), round(distance, 1)),
                        (tl[0], tl[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, COLORS.BLUE, 1)
            cv2.putText(det_marker_image,
                        'pitch:{} yaw:{} roll:{}'.format(round(pitch, 1), round(yaw, 1), round(roll, 1)),
                        (bl[0], bl[1] + 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, COLORS.BLUE, 1)
            
            # extrinsics_matrix = euler_angles2rotation_matrix((-1.0 * pitch, yaw, roll))
            # width, height = measure_by_corners(camera_matrix, extrinsics_matrix, marker_corner[0])
            # cv2.putText(det_marker_image,
            #             'size {} x {}'.format(round(width, 1), round(height, 1)),
            #             (bl[0], bl[1] + 10),
            #             cv2.FONT_HERSHEY_SIMPLEX, 0.4, COLORS.BLUE, 1)
            
            if ctx._aruco_tags_info:
                measure_image[m_y: m_y + m_h, m_x: m_x + m_w] = 127
                cv2.circle(measure_image, (int(c_x), int(c_y)), 2, COLORS.OLIVE, -1)
                cv2.polylines(measure_image, [box], True, COLORS.DARKGRAY, 2)
            
                grid_extrinsics_matrix = ctx._aruco_tags_info.get_grid_extrinsics()
                gr_width, gr_height = measure_by_corners(camera_matrix, grid_extrinsics_matrix, marker_corner[0])
                
                # get different extrinsics for each coner point (big err, due to big diff tvec)
                # aruco_tag_info = ctx._aruco_tags_info.get_nearest_arucotag(tl[0], tl[1])
                # p1 = pixel_to_world(camera_matrix, aruco_tag_info._e, tl[0], tl[1])
                # aruco_tag_info = ctx._aruco_tags_info.get_nearest_arucotag(tr[0], tr[1])
                # p2 = pixel_to_world(camera_matrix, aruco_tag_info._e, tr[0], tr[1])
                # aruco_tag_info = ctx._aruco_tags_info.get_nearest_arucotag(br[0], br[1])
                # p3 = pixel_to_world(camera_matrix, aruco_tag_info._e, br[0], br[1])
                # W, H = p1 - p2, p2 - p3
                # width = round(np.sqrt(W[0]**2 + W[1]**2), 1)
                # height = round(np.sqrt(H[0]**2 + H[1]**2), 1)
                
                center_point = np.mean(marker_corner[0], axis=0, dtype=np.int32)
                aruco_tag_info = ctx._aruco_tags_info.get_nearest_arucotag(center_point[0], center_point[1])
                if aruco_tag_info:
                    width, height = measure_by_corners(camera_matrix, aruco_tag_info._e, marker_corner[0])
                    cv2.putText(measure_image,
                                'grid global: {},{}'.format(round(gr_width, 1), round(gr_height, 1)),
                                (tl[0], tl[1] - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.4, COLORS.BLUE, 1)                
                    cv2.putText(measure_image,
                                'nearest{}: {},{}'.format(aruco_tag_info._t, round(width, 1), round(height, 1)),
                                (bl[0], c_y),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.4, COLORS.BLUE, 1)                
                    cv2.putText(measure_image,
                                'triangle: {},{} {},{}'.format(round(tri_width, 1), round(tri_height, 1), m_w, m_h),
                                (bl[0], bl[1] + 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.4, COLORS.BLUE, 1)                
    return {'det_marker_image': det_marker_image, 'measure_image': measure_image}

def girdaruco_estimate_poses(ctx, image, board, aruco_dict, camera_matrix, dist_coeffs):
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    marker_corners, marker_ids, rejected_points = cv2.aruco.detectMarkers(img_gray, aruco_dict)
    # Refine detected markers
    # Eliminates markers not part of our board, adds missing markers to the board
    marker_corners, marker_ids, rejectedImgPoints, recoveredIds = cv2.aruco.refineDetectedMarkers(
            image = img_gray,
            board = board,
            detectedCorners = marker_corners,
            detectedIds = marker_ids,
            rejectedCorners = rejected_points,
            cameraMatrix = camera_matrix,
            distCoeffs = dist_coeffs)
    image = cv2.aruco.drawDetectedMarkers(image, marker_corners, borderColor=COLORS.CHOCOLATE)
    if marker_ids is None or len(marker_ids) < 20:
        raise RuntimeError('Not found any markers!')
    # Estimate the posture of the gridboard, which is a construction of 3D space
    pose, rvec, tvec = cv2.aruco.estimatePoseBoard(marker_corners, marker_ids, board, camera_matrix, dist_coeffs, None, None)
    if pose:
        # Draw the camera posture calculated from the gridboard
        image = cv2.drawFrameAxes(image, camera_matrix, dist_coeffs, rvec, tvec, 20)
    return image, marker_corners, marker_ids, rvec, tvec

class _Ball(object):
    def __init__(self, x, y, e, t):
        self._x = x
        self._y = y
        self._e = e
        self._t = t
    def __str__(self):
        return f'{self._t}: ({self._x}, {self._y}), {np.round(self._e, 2).tolist()}'

class ArucoTagsInfo(object):
            
    def __init__(self, grid_extrinsics_matrix, xy_map, gap):
        self._grid_extrinsics_matrix = grid_extrinsics_matrix
        self._gap = gap
        self._cx_map_ordered =  collections.OrderedDict(sorted(xy_map.items(), key=lambda kv: kv[0][0]))
        self._cy_map_ordered =  collections.OrderedDict(sorted(xy_map.items(), key=lambda kv: kv[0][1]))

    def get_grid_extrinsics(self):
        return self._grid_extrinsics_matrix 
    
    def get_nearest_arucotag(self, x, y):
        x_min, x_max = x - self._gap, x + self._gap
        y_min, y_max = y - self._gap, y + self._gap
        setx, sety = set(), set()
        for point, ball in self._cx_map_ordered.items():
            if point[0] > x_min and point[0] < x_max:
                setx.add(ball)
        for point, ball in self._cy_map_ordered.items():
            if point[1] > y_min and point[1] < y_max:
                sety.add(ball) 
        balls = list(setx.intersection(sety))
        if len(balls) == 0:
            return None
        k, b = 2 * self._gap, balls[0]
        for ball in balls:
            d = abs(ball._x - x) + abs(ball._y - y)
            if d < k:
                k, b = d, ball
        return b
                 
def detect_object_and_measure(ctx, image, dettype, camera_matrix, dist_coeffs, cross_line_p, cross_line_w):
    result = {}
    cross_line_p = int(cross_line_p * image.shape[0])
    image[cross_line_p: cross_line_p + cross_line_w, :] = 127
    if dettype == 'aruco-6x6': # Detect 6x6 ArUco Marker, [marker size] = [marker id] - 60 (mm)
        dict_id = cv2.aruco.Dictionary_get(cv2.aruco.DICT_6X6_1000)
        result = single_aruco_estimate_pose(ctx, image, dict_id, camera_matrix, dist_coeffs, cross_line_p)
    elif dettype == 'aruco-4x4-26mm':
        dict_id = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_250)
        result = single_aruco_estimate_pose(ctx, image, dict_id, camera_matrix, dist_coeffs, cross_line_p, marker_length=26)
    elif dettype == 'grid-26mm-3mm-9x7-4x4':
        markersX, markersY, marker_size_mm, gap_size_mm = 9, 7, 26, 3
        axis_length = int(0.5 * marker_size_mm)
        dict_id = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_250)
        board = cv2.aruco.GridBoard_create(
            markersX=markersX,
            markersY=markersY,
            markerLength=marker_size_mm,
            markerSeparation=gap_size_mm,
            dictionary=dict_id)
        try:
            img_gird_pose, marker_corners, marker_ids, grid_rvec, grid_tvec = girdaruco_estimate_poses(ctx, image.copy(), board, dict_id, camera_matrix, dist_coeffs)
            result['grid_pose'] = img_gird_pose
            if len(marker_corners) == markersX * markersY:
                rvecs, tvecs, marker_points = cv2.aruco.estimatePoseSingleMarkers(marker_corners, marker_size_mm, camera_matrix, dist_coeffs)
                xy_map, length = {}, -1
                for mc, mid, rvec, tvec in zip(marker_corners, marker_ids, rvecs, tvecs):
                    extrinsics_matrix = np.concatenate([cv2.Rodrigues(rvec)[0], tvec.T], axis=1) 
                    if length < 0:
                        length = max(abs(mc[0][0][0] - mc[0][2][0]), abs(mc[0][0][1] - mc[0][2][1]))
                    center = tuple(np.mean(mc[0], axis=0, dtype=np.int32).tolist())
                    width, hight = measure_by_corners(camera_matrix, extrinsics_matrix, mc[0])
                    
                    (cx, cy), (w, h), angle = cv2.minAreaRect(mc[0])
                    tl_x, tl_y, w, h = int(cx - 0.5 * w), int(cy - 0.5 * h), int(w), int(h)
                    image[tl_y:tl_y + h, tl_x:tl_x + w] = 255
                    
                    cv2.drawFrameAxes(image, camera_matrix, dist_coeffs, rvec, tvec, axis_length)
                    cv2.putText(image, f'{round(width, 1)}', (center[0] - 12, center[1] - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.3, COLORS.BLACK, 1)
                    cv2.putText(image, f'{round(hight, 1)}', (center[0] - 12, center[1] + 8), cv2.FONT_HERSHEY_SIMPLEX, 0.3, COLORS.BLACK, 1)
                    if ctx._aruco_tags_info is None:
                        xy_map[center] = _Ball(center[0], center[1], extrinsics_matrix, mid[0])
                if ctx._aruco_tags_info is None:
                    grid_extrinsics_matrix = np.concatenate([cv2.Rodrigues(grid_rvec)[0], grid_tvec], axis=1) 
                    ctx._aruco_tags_info = ArucoTagsInfo(grid_extrinsics_matrix, xy_map, length)
            result['markers'] = image
        except Exception as err:
            ctx.logger(f'{err}: {traceback.format_exc()}', clear=1)
            time.sleep(0.3)
            # raise
        
    elif dettype == 'black_edge_carton':
        if ctx._aruco_tags_info is None:
            cv2.putText(image, 'no extrinsics matrix', (50, 250), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS.LIGHT_BLUE, 1)
        
        img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        img_blur = cv2.GaussianBlur(img_gray, (5, 5), 0)
        img_edged = cv2.Canny(img_blur, 50, 100)
        img_edged = cv2.dilate(img_edged, None, iterations=1)
        img_edged = cv2.erode(img_edged, None, iterations=1)
        contours = cv2.findContours(img_edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
        # cv2.drawContours(image, contours, -1, color=(0, 0, 255), thickness=4)
        # for cnt in contours:
        #     area = cv2.contourArea(cnt)
        #     if area > 1000 and area < 20000:
        #         rect = cv2.minAreaRect(cnt)
        #         (x, y), (w, h), angle = rect
        #         box = cv2.boxPoints(rect)
        #         box = np.int0(box)
        #         cv2.circle(image, (int(x), int(y)), 10, (0, 0, 255), -1)
        #         cv2.polylines(image, [box], True, (100, 10, 0), 2)
    elif dettype == 'tagboard':
        if ctx._aruco_tags_info is None:
            cv2.putText(image, 'no extrinsics matrix', (50, 250), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS.LIGHTBLUE, 1)
            
        img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        img_blur = cv2.GaussianBlur(img_gray, (3, 3), 0)
        img_edged = cv2.Canny(img_blur, 50, 100)
        # img_edged = cv2.dilate(img_edged, None, iterations=1)
        # img_edged = cv2.erode(img_edged, None, iterations=1)
        contours = cv2.findContours(img_edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
        cv2.drawContours(image, contours, -1, color=(0, 0, 255), thickness=4)
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area < 600 and area > 200:
                rect = cv2.minAreaRect(cnt)
                (x, y), (w, h), angle = rect
                if w > h:
                    rate = w / h
                else:
                    rate = h / w
                if rate < 2:
                    continue
                box = np.int0(cv2.boxPoints(rect))
                cv2.circle(image, (int(x), int(y)), 2, COLORS.RED, -1)
                cv2.polylines(image, [box], True, COLORS.BLUE, 1)                
                try:
                    aruco_tag_info = ctx._aruco_tags_info.get_nearest_arucotag(x, y)
                    if aruco_tag_info:
                        width, height = measure_by_corners(camera_matrix, aruco_tag_info._e, box)
                        cv2.putText(image, '%.2f, %.2f' % (width, height), box[0], cv2.FONT_HERSHEY_SIMPLEX, 0.4, COLORS.LIGHTBLUE, 1)
                        result['image'] = image
                except:
                    pass
        
    return result

# def foreach_read_frame(source):
#     camera = cv2.VideoCapture(source)
#     if not camera.isOpened():
#         raise RuntimeError(f'Cannot open video: {camera_source}')
#     frame_idx = 0
#     while True:
#         ret, frame_bgr = camera.read()
#         if not ret:
#             raise RuntimeError('Camera read error!')
#         frame_idx += 1
#         yield frame_bgr

### Calibration By ChessBoard

In [7]:
def start_calibrate_by_chessboard(
    ctx, w_btn_source, camera_source, squares_x, squares_y, square_size, win_size, term_iters, term_eps,
    display_items, sample_size, use_file, w_resolution, w_cross_line_p, w_cross_line_w, 
    w_det_objtype, w_sample_list,  w_cam_frame):
    
    ctx.logger(f'start: {squares_x}, {squares_y}, {square_size}mm, {sample_size}', clear=1)
    
    # Object points for a chessboard
    pattern_size = (squares_x - 1, squares_y - 1)
    objp = np.zeros((np.prod(pattern_size), 3), np.float32)
    objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)
    objp = objp * square_size
        
    win_size, zero_zone = (win_size, win_size), (-1, -1)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, term_iters, term_eps)
    
    def _init_load_images(img_dir, online=False):
        obj_points, img_points = [], []
        samples = []
        options = [('Camera', 0)]
        state = CalibrateState.COLLECT
        if not online and os.path.exists(img_dir):
            image_list = glob.glob(f'{img_dir}/*.png')
            assert len(image_list) > 3, 'Not enough chess board images'
            display_images = {}
            for path in image_list:
                name = os.path.basename(path)
                img = cv2.imread(path)
                # img = cv2.flip(img, 1)
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                found, corners = cv2.findChessboardCorners(gray, pattern_size)
                better_chess_corners = cv2.cornerSubPix(gray, corners, win_size, zero_zone, criteria)
                obj_points.append(objp)
                img_points.append(better_chess_corners)
                display_images['orig_corners'] = cv2.drawChessboardCorners(img.copy(), pattern_size, corners, found)
                display_images['best_corners'] = cv2.drawChessboardCorners(img.copy(), pattern_size, better_chess_corners, found)
                options.append((name, int(name.split('.')[0])))
                samples.append(nbeasy_widget_display(display_images))
            w_sample_list.options = sorted(options, key=lambda item: item[1])
            state = CalibrateState.CALIBRATE
        else:
            shutil.rmtree(img_dir, ignore_errors=True)
            ctx.logger(f'rm {img_dir}')
        os.makedirs(img_dir, exist_ok=True)
        ctx._out_dir = img_dir
        return state, obj_points, img_points, samples, options
        
    def _video_capture():
        w_btn_source.disabled = True
        out_dir = f'out/chessboard/{squares_x}x{squares_y}_{square_size}_{w_resolution.value[0]}'
        try:
            state, obj_points, img_points, samples, options = _init_load_images(out_dir, sample_size is not None)
                
            camera = cv2.VideoCapture(camera_source)
            if not camera.isOpened():
                raise RuntimeError(f'Cannot open video: {camera_source}')
                
            # frame_width = int(camera.get(cv2.CAP_PROP_FRAME_WIDTH))
            # frame_height = int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT))
            frame_irate = int(camera.get(cv2.CAP_PROP_FPS))
            
            frame_width, frame_height = w_resolution.value
            camera.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
            camera.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_height)
            w_cam_frame.width, w_cam_frame.height = str(frame_width), str(frame_height)
            
            ctx.logger(f'width:{frame_width} height:{frame_height} display: {w_cam_frame.width} resoluton: {w_resolution.value}')

            ctx.camera = camera
            ctx.camera_is_running = camera.isOpened()

            ctx._aruco_tags_info = None
            camera_matrix, dist_coeffs = None, None

            frame_idx, iter_idx = 0, 0
            
            show_raw, show_undist = 'raw' in display_items, 'undist' in display_items
            while ctx.camera_is_running:
                if w_sample_list.value != 0:
                    w_cam_frame.value = samples[w_sample_list.value - 1]
                    time.sleep(1)
                    continue
                if frame_width != w_resolution.value[0]:
                    frame_width, frame_height = w_resolution.value
                    camera.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
                    camera.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_height)
                    w_cam_frame.width, w_cam_frame.height = str(frame_width), str(frame_height)
                    out_dir = f'out/chessboard/{squares_x}x{squares_y}_{square_size}_{w_resolution.value[0]}'
                    state, obj_points, img_points, samples, options = _init_load_images(out_dir, False)
                    frame_idx, iter_idx = 0, 0
                    ctx.logger(f'frame_width:{frame_width} frame_height:{frame_height} display: {w_cam_frame.width}')
                    
                ret, frame_bgr = camera.read()
                if not ret:
                    break
                # frame_bgr = cv2.flip(frame_bgr, 1)
                display_frames = {'raw': frame_bgr}  if show_raw else {}
                frame_gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
                if state == CalibrateState.COLLECT:
                    found, corners = cv2.findChessboardCorners(frame_gray, pattern_size)
                    if found:
                        # Find better sub pix position for the corners in the roof corners neighbourhood
                        better_chess_corners = cv2.cornerSubPix(frame_gray, corners, win_size, zero_zone, criteria)
                        display_frames['orig_corners'] = cv2.drawChessboardCorners(frame_bgr.copy(), pattern_size, corners, found)
                        display_frames['best_corners'] = cv2.drawChessboardCorners(frame_bgr.copy(), pattern_size, better_chess_corners, found)

                        if frame_idx % frame_irate == 0:
                            obj_points.append(objp)
                            img_points.append(better_chess_corners)
                            iter_idx += 1

                            box = cv2.boxPoints(cv2.minAreaRect(better_chess_corners.reshape(-1, 2)))
                            cv2.drawContours(display_frames['best_corners'], [np.int0(box)], 0, COLORS.ORANGE, 2)

                            # Draw and display the corners
                            options.append((str(iter_idx) + '.png', iter_idx))
                            samples.append(io.BytesIO(cv2.imencode('.png', display_frames['best_corners'])[1]).getvalue())
                            if iter_idx == sample_size:
                                state = CalibrateState.CALIBRATE
                                w_sample_list.options = sorted(options, key=lambda item: item[1])
                            cv2.imwrite(f'{out_dir}/{iter_idx}.png', frame_bgr)
                            print(f'{out_dir}/{iter_idx}.png')
                        cv2.putText(display_frames['best_corners'], f'Count: {iter_idx}', (200, 200), cv2.FONT_HERSHEY_SIMPLEX, 1.2, COLORS.LIGHTBLUE, 2)
                elif state == CalibrateState.CALIBRATE and not use_file:
                    error, camera_matrix, dist_coeffs = start_calibrate_camera_with_good_flags(
                        ctx, functools.partial(
                            cv2.calibrateCameraExtended, objectPoints=obj_points, imagePoints=img_points),
                        img_size=frame_gray.shape[::-1],
                    )

                    save_calibration_intrinsics(camera_matrix, dist_coeffs, out_dir)
                    
                    # obj_points, img_points = obj_points[-1], img_points[-1]
                    # # Calculating extrinsics by last imgage (rotation vector)
                    # retval, rvec, tvec = cv2.solvePnP(obj_points, img_points, camera_matrix, dist_coeffs)                    
                    # # rvec, tvec = np.mean(np.array(rvecs), axis=0), np.mean(np.array(tvecs), axis=0)
                    # # transform rotation vector to ratation matrix
                    # rotation_matrix, _ = cv2.Rodrigues(rvec)
                    # extrinsics_matrix = np.concatenate([rotation_matrix, tvec], 1) 
                    
                    # calculate the re-project points error.  == reval
                    # mean_error = 0
                    # for obj_point, img_point, rvec, tvec in zip(obj_points, img_points, rvecs, tvecs):
                    #     img_point_proj, _ = cv2.projectPoints(obj_point, rvec, tvec, camera_matrix, dist_coeffs)
                    #     error = cv2.norm(img_point, img_point_proj, cv2.NORM_L2) / len(img_point_proj)
                    #     mean_error += error
                    # ctx.logger('total error: {}'.format(mean_error / len(obj_points)))
             
                    state = CalibrateState.COMPLETED
                else: 
                    # load from file
                    if camera_matrix is None and use_file:
                        camera_matrix, dist_coeffs = load_calibration_intrinsics(out_dir)
                        
                    if show_undist:
                        display_frames['undist'] = cv2.undistort(frame_bgr.copy(), camera_matrix, dist_coeffs)
                        
                    result = detect_object_and_measure(ctx, frame_bgr.copy(), w_det_objtype.value, camera_matrix, dist_coeffs, w_cross_line_p.value, w_cross_line_w.value)
                    for title, image in result.items():
                        display_frames[title] = image

                # display image
                nbeasy_widget_display(display_frames, w_cam_frame)
                frame_idx += 1

        except Exception as err:
            ctx.logger(f'{err}: {traceback.format_exc()}')
        finally:
            w_btn_source.disabled = False
            ctx.camera.release()
            ctx.camera_is_running = False
            ctx.camera = None

    camera_thread = threading.Thread(target=_video_capture, name='camera')
    camera_thread.daemon = True
    camera_thread.start()

stop_calibrate_by_chessboard = stop_camera_calibrate

### Calibration By CharucoBoard

In [8]:
def start_calibrate_by_charucoboard(
    ctx, w_btn_source, camera_source, squares_x, squares_y, square_size, win_size, marker_size, 
    aruco_id, term_iters, term_eps, display_items, sample_size, use_file, w_resolution,
    w_cross_line_p, w_cross_line_w, w_det_objtype, w_sample_list, w_cam_frame):
    
    ctx.logger(f'start: {squares_x}, {squares_y}, {square_size}mm, {marker_size}mm, {aruco_id}, {sample_size}, {use_file}', clear=1)
    
    charuco_dict = cv2.aruco.getPredefinedDictionary(aruco_id)
    charuco_board = cv2.aruco.CharucoBoard_create(squares_x, squares_y, square_size, marker_size, charuco_dict)
    
    pattern_size = (squares_x, squares_y)
    win_size, zero_zone = (win_size, win_size), (-1, -1)
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, term_iters, term_eps)
        
    def _detect_charuco_corners(frame_gray, charuco_board, charuco_dict, win_size, zero_zone, criteria, lowlimit):
        # corner detection
        marker_corners, marker_ids, rejected_points = cv2.aruco.detectMarkers(frame_gray, charuco_dict)
        if marker_corners is not None and len(marker_corners) > 0:
            # corner refine 
            # marker_corners, marker_ids, refusd, recoverd = cv2.aruco.refineDetectedMarkers(
            #     frame_gray, charuco_board, marker_corners, marker_ids, rejectedCorners=rejected_points) 
            # corner interpolation (get charuco corners and ids from detected aruco markers)
            retval, charuco_corners, charuco_ids = cv2.aruco.interpolateCornersCharuco(marker_corners, marker_ids, frame_gray, charuco_board)
            if charuco_corners is not None and len(charuco_corners) > lowlimit:
                if win_size:
                    better_charuco_corners = cv2.cornerSubPix(frame_gray, charuco_corners, win_size, zero_zone, criteria)
                else:
                    better_charuco_corners  = charuco_corners
                return retval, marker_corners, marker_ids, charuco_corners, better_charuco_corners, charuco_ids 
        return None, None, None, None, None, None
    
    def _init_load_images(img_dir, online=False):
        samples = []
        options = [('Camera', 0)]
        state = CalibrateState.COLLECT
        if not online and os.path.exists(img_dir):
            image_list = glob.glob(f'{img_dir}/*.png')
            assert len(image_list) > 10, 'Not enough chessaruco board images'
            for path in image_list:
                name = os.path.basename(path)
                img = cv2.imread(path)
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                retval, marker_corners, marker_ids, charuco_corners, better_charuco_corners, charuco_ids = _detect_charuco_corners(
                        gray, charuco_board, charuco_dict, win_size, zero_zone, criteria)
                if retval:
                    all_charuco_corners.append(better_charuco_corners)
                    all_charuco_ids.append(charuco_ids)
                    cv2.aruco.drawDetectedCornersCharuco(img, better_charuco_corners, charuco_ids)
                    options.append((name, int(name.split('.')[0])))
                    samples.append(io.BytesIO(cv2.imencode('.png', img)[1]).getvalue())
            w_sample_list.options = sorted(options, key=lambda item: item[1])
            state = CalibrateState.CALIBRATE
        else:
            shutil.rmtree(img_dir, ignore_errors=True)
        os.makedirs(img_dir, exist_ok=True)
        ctx._out_dir = img_dir
        return state, all_charuco_corners, all_charuco_ids, samples, options
    
    def _video_capture():
        w_btn_source.disabled = True
        result = {}
        all_charuco_corners, all_charuco_ids = [], []
        out_dir = f'out/chessaruco/{squares_x}x{squares_y}_{square_size}_{marker_size}_{w_resolution.value[0]}'
            
        try:
            state, all_charuco_corners, all_charuco_ids, samples, options = _init_load_images(out_dir, sample_size is not None)
                
            camera = cv2.VideoCapture(camera_source)
            if not camera.isOpened():
                raise RuntimeError(f'Cannot open video: {camera_source}')
                
            # frame_width = int(camera.get(cv2.CAP_PROP_FRAME_WIDTH))
            # frame_height = int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT))
            
            frame_irate = int(camera.get(cv2.CAP_PROP_FPS))
            frame_width, frame_height = w_resolution.value
            camera.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
            camera.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_height)
            w_cam_frame.width, w_cam_frame.height = str(frame_width), str(frame_height)
            
            ctx.logger(f'frame_width:{frame_width} frame_height:{frame_height} display: {w_cam_frame.width}')

            ctx.camera = camera
            ctx.camera_is_running = camera.isOpened()
            
            ctx._aruco_tags_info = None
            camera_matrix, dist_coeffs = None, None,
            
            lowlimit = int(0.5 * squares_x * squares_y)
            frame_idx, iter_idx = 0, 0
            
            show_raw, show_undist = 'raw' in display_items, 'undist' in display_items
            while ctx.camera_is_running:
                if w_sample_list.value != 0:
                    if w_cam_frame.layout.width != frame_width:
                        w_cam_frame.layout.width = f'{frame_width}px'
                        w_cam_frame.layout.height = f'{frame_height}px'
                    w_cam_frame.value = samples[w_sample_list.value]
                    time.sleep(1)
                    continue
                if frame_width != w_resolution.value[0]:
                    frame_width, frame_height = w_resolution.value
                    camera.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
                    camera.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_height)
                    w_cam_frame.width, w_cam_frame.height = str(frame_width), str(frame_height)
                    out_dir = f'out/chessaruco/{squares_x}x{squares_y}_{square_size}_{marker_size}_{w_resolution.value[0]}'
                    state, all_charuco_corners, all_charuco_ids, samples, options = _init_load_images(out_dir, False)
                    frame_idx, iter_idx = 0, 0
                    ctx.logger(f'frame_width:{frame_width} frame_height:{frame_height} display: {w_cam_frame.width}')
                ret, frame_bgr = camera.read()
                if not ret:
                    break
                # frame_bgr = cv2.flip(frame_bgr, 1)
                display_frames = {'raw': frame_bgr}  if show_raw else {}
                frame_gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
                if state == CalibrateState.COLLECT:
                    retval, marker_corners, marker_ids, charuco_corners, better_charuco_corners, charuco_ids = _detect_charuco_corners(
                        frame_gray, charuco_board, charuco_dict, win_size, zero_zone, criteria, lowlimit)
                    
                    if retval:
                        display_frames['markers'] = cv2.aruco.drawDetectedMarkers(frame_bgr.copy(), marker_corners, marker_ids)                        
                        display_frames['orig_corners'] = cv2.aruco.drawDetectedCornersCharuco(frame_bgr.copy(), charuco_corners, charuco_ids)
                        display_frames['best_corners'] = cv2.aruco.drawDetectedCornersCharuco(frame_bgr.copy(), better_charuco_corners, charuco_ids)

                        if frame_idx % frame_irate == 0:
                            # Draw and Display markers and corners
                            # objp = charuco_board.chessboardCorners[all_charuco_ids.flatten()]
                            all_charuco_corners.append(better_charuco_corners)
                            all_charuco_ids.append(charuco_ids)
                            iter_idx += 1

                            box = cv2.boxPoints(cv2.minAreaRect(better_charuco_corners.reshape(-1, 2)))
                            cv2.drawContours(display_frames['best_corners'], [np.int0(box)], 0, COLORS.ORANGE, 2)

                            options.append((str(iter_idx) + '.png', iter_idx))
                            samples.append(io.BytesIO(cv2.imencode('.png', display_frames['best_corners'])[1]).getvalue())
                            if iter_idx == sample_size:
                                sample_size = None
                                state = CalibrateState.CALIBRATE
                                w_sample_list.options = sorted(options, key=lambda item: item[1])
                            cv2.imwrite(f'{out_dir}/{iter_idx}.png', frame_bgr)
                        cv2.putText(display_frames['best_corners'], f'Count: {iter_idx}', (200, 200), cv2.FONT_HERSHEY_SIMPLEX, 1.2, COLORS.GOLDEN, 2)
                elif state == CalibrateState.CALIBRATE and not use_file:
                    error, camera_matrix, dist_coeffs = start_calibrate_camera_with_good_flags(
                        ctx, functools.partial(
                            cv2.aruco.calibrateCameraCharucoExtended,
                            charucoCorners=all_charuco_corners, charucoIds=all_charuco_ids,
                            board=charuco_board),
                        img_size=frame_gray.shape[::-1]
                    )
                        
                    save_calibration_intrinsics(camera_matrix, dist_coeffs, out_dir)
                    
                    # total_error = 0.0
                    # for k, (charuco_corners, c_ids, rvec, tvec) in enumerate(zip(all_charuco_corners, all_charuco_ids, rvecs, tvecs)):
                    #     obj_points, img_points = cv2.aruco.getBoardObjectAndImagePoints(charuco_board, charuco_corners, c_ids)
                    #     prj_points, _ = cv2.projectPoints(obj_points, rvec, tvec, camera_matrix, dist_coeffs)
                    #     error = cv2.norm(img_points, prj_points, cv2.NORM_L2) / len(obj_points)
                    #     total_error += error
                    # ctx.logger(f'mean total error: {total_error / len(all_charuco_corners)}')
                    state = CalibrateState.COMPLETED
                else: 
                    # load from file
                    if camera_matrix is None and use_file:
                        camera_matrix, dist_coeffs = load_calibration_intrinsics(out_dir)
                     
                    if show_undist:
                        display_frames['undist'] = cv2.undistort(frame_bgr.copy(), camera_matrix, dist_coeffs)
                        
                    result = detect_object_and_measure(ctx, frame_bgr.copy(), w_det_objtype.value, camera_matrix, dist_coeffs, w_cross_line_p.value, w_cross_line_w.value)
                    for title, image in result.items():
                        display_frames[title] = image
                    
                # display image
                nbeasy_widget_display(display_frames, w_cam_frame)
                frame_idx += 1
                
        except Exception as err:
            ctx.logger(f'{err}: {traceback.format_exc()}')
        finally:
            w_btn_source.disabled = False
            ctx.camera.release()
            ctx.camera_is_running = False
            ctx.camera = None
        
    camera_thread = threading.Thread(target=_video_capture, name='camera')
    camera_thread.daemon = True
    camera_thread.start()

stop_calibrate_by_charucoboard = stop_camera_calibrate

### Other Events

In [9]:
def reset_grid_aurco_tag(ctx, w_btn_source):
    ctx.logger('reset_grid_aurco_tag', clear=1)
    if getattr(ctx, '_aruco_tags_info'):
        ctx._aruco_tags_info = None
        
def save_grid_aurco_tag(ctx, w_btn_source):
    ctx.logger('save_grid_aurco_tag', clear=1)
    if getattr(ctx, '_aruco_tags_info'):
        with open(f'{ctx._out_dir}/grid_aruco_tags_info.pkl', 'wb') as f:
            pickle.dump(ctx._aruco_tags_info, f)
        
def load_grid_aurco_tag(ctx, w_btn_source):
    ctx.logger('load_grid_aurco_tag', clear=1)
    with open(f'{ctx._out_dir}/grid_aruco_tags_info.pkl', 'rb') as f:
        ctx._aruco_tags_info = pickle.load(f)

### Template Schema

In [21]:
def nbeasy_chessboard_choice_objs(nrow, ncol, square_size, width=300, ro=True):
    return {
        'type': 'H',
        'objs': [
            nbeasy_widget_int('cfg.chessboard_squares_x', 'Squares X', nrow, width=width, readonly=ro),
            nbeasy_widget_int('cfg.chessboard_squares_y', 'Squares Y', ncol, width=width, readonly=ro),
            nbeasy_widget_int('cfg.chessboard_square_size', 'Square Size(mm)', square_size, width=width, readonly=ro)        
        ]
    }

def nbeasy_charucoboard_choice_objs(nrow, ncol, square_size, marker_size, aruco_dict_id, width=300, ro=True):
    enums = [
        ('4x4', cv2.aruco.DICT_4X4_1000),
        ('5x5', cv2.aruco.DICT_5X5_1000),
    ]
    return {
        'type': 'H',
        'objs': [
            nbeasy_widget_int('cfg.charucoboard_squares_x', 'Squares X', nrow, width=width, readonly=ro),
            nbeasy_widget_int('cfg.charucoboard_squares_y', 'Squares Y', ncol, width=width, readonly=ro),
            nbeasy_widget_int('cfg.charucoboard_square_size', 'Square Size(mm)', square_size, width=width, readonly=ro),
            nbeasy_widget_float('cfg.charucoboard_marker_size', 'Marker Size CM', marker_size, width=width, readonly=ro),
            nbeasy_widget_stringenum(
                'cfg.charucoboard_aruco_dict', 'ArUco Dict', default=aruco_dict_id,
                enums=enums,
                width=width,
                readonly=ro
            )
        ]
    }

def nbeasy_common_shared_by_tagtype(tag):
    return {
        'type': 'V',
        'objs': [
            {
                'type': 'H',
                'objs': [
                    nbeasy_widget_int(f'cfg.{tag}_win_size', 'Win Size', default=5),
                    nbeasy_widget_int(f'cfg.{tag}_term_iters', 'Term Iters', default=500),
                    nbeasy_widget_float(f'cfg.{tag}_term_eps', 'Term EPS', default=0.0001),
                ]
            },
            {
                'type': 'V',
                'name': 'Camera',
                'objs': [
                    {
                        'type': 'H',
                        'objs': [
                            nbeasy_widget_togglebuttons(
                                f'cfg.{tag}_resolution', ' ',  default=0,
                                enums=[('SD ', [640, 480]), ('HD ', [1080, 720])],
                                style='info', icons=['camera', 'camera']),
                            nbeasy_widget_button(f'cfg.__btn_{tag}_start_camera', 'Start', width=200, style='success'),
                            nbeasy_widget_button(f'cfg.__btn_{tag}_stop_camera', 'Stop', width=200, style='success'),
                        ],
                        'justify_content': 'center'
                    },
                ],
                'align_items': 'center'
            },
        ]
    }

def nbeasy_start_stop_camera(tag):
    if tag == "chessboard":
        start_handler = start_calibrate_by_chessboard
        stop_handler = stop_calibrate_by_chessboard
    elif tag == "charucoboard":
        start_handler = start_calibrate_by_charucoboard
        stop_handler = stop_calibrate_by_charucoboard
    else:
        raise
        
    return [{
        'handler': start_handler,
        'params': {
            'sources': [f'cfg.__btn_{tag}_start_camera'],
            'targets': [
                'cfg.camera_source:value',
                f'cfg.{tag}_squares_x:value',
                f'cfg.{tag}_squares_y:value',
                f'cfg.{tag}_square_size:value',
                f'cfg.{tag}_win_size:value',
            ] + (
                [
                    'cfg.charucoboard_marker_size:value',
                    'cfg.charucoboard_aruco_dict:value',
                ] if tag == "charucoboard" else []
            ) + [
                f'cfg.{tag}_term_iters:value',
                f'cfg.{tag}_term_eps:value',
                'cfg.display_items:value',
                'cfg.sample_size:value',
                'cfg.use_file:value',
                f'cfg.{tag}_resolution',
                'cfg.cross_line_p',
                'cfg.cross_line_w',
                'cfg.det_object_type',
                'cfg.sample_list',
                '__cfg.video_frame'
            ]
        }
    },
    {
        'handler': stop_handler,
        'params': {
            'sources': [f'cfg.__btn_{tag}_stop_camera'],
            'targets': [
            ]
        }
    }]

schema = {
    'type': 'page',
    'objs': [
        {
            'type': 'H',
            'name': 'Common',
            'objs': [
                    nbeasy_widget_stringenum(
                        'cfg.camera_source', 'Camera Source', default=1,
                        enums=[('usb video 0', 0), ('usb video 1', 1), ('office stream 0', 'rtmp://101.42.139.3:31935/live/48b02d5bd01d?vhost=seg.300s')],
                        width=300),
                    nbeasy_widget_booltrigger(f'_cfg.is_online', 'Online', default=False, triggers=[
                        {
                            'type': 'H',
                            'objs': [
                                nbeasy_widget_bool(f'cfg.use_file', 'Use File', default=True)
                            ]
                        },
                        {
                            'type': 'H',
                            'objs': [
                                nbeasy_widget_int(f'cfg.sample_size', 'Sample Size', 32, width=333)
                            ]
                        }
                    ], width=200),
            ]
        },
        {
            'type': 'H',
            'name': 'Display',
            'objs': [
                nbeasy_widget_multiselect(f'cfg.display_items', 'Select',  enums=[
                    ('Camera Raw Frame', 'raw'),
                    ('Compensate Distortion', 'undist')
                ], width=300)
            ]
        },
        {
            'type': 'tab',
            'objs': [
                {
                    'type': 'V',
                    'name': 'ChessBoard',
                    'objs': [
                        nbeasy_widget_stringenumtrigger(
                            '_cfg.chessboard_choice', 'Board Choise', default=1,
                            enums = ['A4-15mm-11x8', 'A4-30mm-9x7', 'A3-25mm-16x11'],
                            triggers = [
                                nbeasy_chessboard_choice_objs(11, 8, 15),
                                nbeasy_chessboard_choice_objs(9, 7, 30),
                                nbeasy_chessboard_choice_objs(16, 11, 25),
                            ],
                            width=333),
                        nbeasy_common_shared_by_tagtype('chessboard'),
                    ],
                }, # end chessboard
                {
                    'type': 'V',
                    'name': 'CharucoBoard',
                    'objs': [
                        nbeasy_widget_stringenumtrigger(
                            '_cfg.charucoboard_choice', 'Board Choise', default=2,
                            enums = ['A3-20mm-16mm-19x13-4x4', 'A4-30mm-23mm-9x6-4x4', 'A4-25mm-19mm-11x8-5x5', 'A4-15mm-12mm-11x8-5x5'],
                            triggers = [
                                nbeasy_charucoboard_choice_objs(19, 13, 20, 16, 0),
                                nbeasy_charucoboard_choice_objs(9, 6, 30, 23, 0),
                                nbeasy_charucoboard_choice_objs(11, 8, 25, 19, 1),
                                nbeasy_charucoboard_choice_objs(11, 8, 15, 12, 1),
                            ],
                            width=333),
                        nbeasy_common_shared_by_tagtype('charucoboard'),
                    ],
                }, # end charucoboard
            ], # end tab objs
        },
        {
            'type': 'V',
            'objs': [
                nbeasy_widget_image('__cfg.video_frame', 'Frame', '', width=640, height=480),
                {
                    'type': 'H',
                    'objs': [
                        nbeasy_widget_stringenumtrigger(
                            'cfg.det_object_type', 'Detect Object', default=0,
                            enums=['aruco-6x6', 'grid-26mm-3mm-9x7-4x4', 'aruco-4x4-26mm', 'black_edge_carton', 'tagboard'],
                            triggers = [
                                {
                                },
                                {
                                    'objs': [
                                        {
                                            'type': 'H',
                                            'objs': [
                                                nbeasy_widget_button('cfg.__btn_reset_tags', 'Reset Tags', width=200, style='success'),
                                                nbeasy_widget_button('cfg.__btn_save_tags', 'Save Tags', width=200, style='success'),
                                            ]
                                        }
                                    ],
                                },
                                {
                                },
                                {
                                },
                                {
                                    'objs': [
                                        nbeasy_widget_button('cfg.__btn_load_tags', 'Load Tags', width=200, style='success'),
                                    ]
                                }
                            ],
                            width=333),
                        nbeasy_widget_float('cfg.cross_line_p', 'Line Position', default=0.99),
                        nbeasy_widget_int('cfg.cross_line_w', 'Line Width', default=3),
                        nbeasy_widget_stringenum(
                            'cfg.sample_list', 'Sample List', default=0,
                            enums=[('Camera', 0)],
                            width=320)
                    ],
                    'justify_content': 'center'
                }
            ],
            'align_items': 'center'
        }, # end video
    ], # end page objs
    'evts': [
        {
            'type': 'jsdlink',
            'objs': [
                {
                    'source': 'cfg.__btn_chessboard_start_camera:disabled',
                    'target': 'cfg.__btn_charucoboard_start_camera:disabled'
                }
            ]
        },
        {
            'type': 'onclick',
            'objs': [
                *nbeasy_start_stop_camera('chessboard'),
                *nbeasy_start_stop_camera('charucoboard'),
                {
                    'handler': reset_grid_aurco_tag,
                    'params': {
                        'sources': ['cfg.__btn_reset_tags'],
                        'targets': []
                    }
                },
                {
                    'handler': save_grid_aurco_tag,
                    'params': {
                        'sources': ['cfg.__btn_save_tags'],
                        'targets': []
                    }
                },
                {
                    'handler': load_grid_aurco_tag,
                    'params': {
                        'sources': ['cfg.__btn_load_tags'],
                        'targets': []
                    }
                },
            ]
           
        }
    ], # end event
}

if g_ctx:
    stop_camera_calibrate(g_ctx)
g_ctx = nbeasy_schema_parse(schema, debug=True, border=False)

Box(children=(Box(children=(VBox(children=(HTML(value="<b><font color='black'>Common :</b>"), HBox(children=(D…

## Binocular: Stereo Camera Calibration

### Utils  Define

In [79]:
def _get_cameras():
    prefix = '/sys/class/video4linux/'
    cameras = []
    for dev in os.listdir(prefix):
        with open(f'{prefix}/{dev}/name', 'r') as f:
            name = f.readline().strip().split(':')[0]
            ty = 'monocular'
            if '3D' in name:
                ty = 'binocular'
            cameras.append((name, (f'/dev/{dev}', ty)))
    return cameras
        
    
VIDEO_MAX_SIZE = 10000

class StereoCamera(object):
    def __init__(self, source, is_3d = False):
        self.video_source = source
        self.video_capture = None
        self.is_running = False
        self._is_3d = is_3d
        # self.grabbed, self.frame = False, None
        # self.read_lock, self.read_thread = threading.Lock(), None
        
    def is_3d(self):
        return self._is_3d
        
    # def start(self):
    #     if self.is_running:
    #         return
    #     self.grabbed, self.frame = self.video_capture.read()
    #     if self.grabbed:
    #         self.is_running = True
    #         self.read_thread = threading.Thread(target=self._update_frame)
    #         self.read_thread.start()
    #         
    # def stop(self):
    #     if self.is_running:
    #         self.is_running = False 
    #         if self.video_capture:
    #             self.video_capture.release()
    #         if self.read_thread:
    #             self.read_thread.join()
    #     self.video_capture, self.read_thread = None, None
    #     
    # def _update_frame(self):
    #     try:
    #         while self.is_running:
    #             grabbed, frame = self.video_capture.read()
    #             with self.read_lock:
    #                 self.grabbed = grabbed
    #                 self.frame = frame
    #     finally:
    #         self.stop()
    #         
    # def read(self):
    #     with self.read_lock:
    #         grabbed = self.grabbed
    #         frame = self.frame.copy()
    #     return grabbed, frame
    # 
    # def read_left_right(self):
    #     with self.read_lock:
    #         frame = self.frame.copy()
    #         grabbed = self.grabbed
    #         frameL = frame[:, :int(self.width / 2), :]
    #         frameR = frame[:, int(self.width / 2):, :]
    #     return grabbed, frameL, frameL
    
    def open(self):
        video_capture = cv2.VideoCapture(self.video_source)
        if not video_capture.isOpened():
            raise RuntimeError(f'Cannot open {self.video_source}')
            
        if self._is_3d:
            video_capture.set(cv2.CAP_PROP_FRAME_WIDTH, VIDEO_MAX_SIZE) # 2560
            video_capture.set(cv2.CAP_PROP_FRAME_HEIGHT, VIDEO_MAX_SIZE) # 960
            width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
            if width != 2560:
                video_capture.release()
                raise RuntimeError(f'{self.video_source} is not stereo camera!')
            video_capture.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        else:
            video_capture.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        video_capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
            
        self.width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.fps = int(video_capture.get(cv2.CAP_PROP_FPS))
        
        self.video_capture = video_capture
        self.is_running = True
        return self.video_capture.read()[1].shape
        
    def close(self):
        self.is_running = False
        for _ in range(10):
            if self.video_capture is None:
                break
            time.sleep(0.3)
    
    def read(self, flip=None):
        try:
            idx = 0
            while self.is_running:
                grabbed, frame = self.video_capture.read()
                if not grabbed:
                    raise RuntimeError('Video read error!')
                idx += 1
                
                if flip > -2:
                    frame = cv2.flip(frame, flip)
                if self._is_3d:
                    # 分离左右摄像头
                    frameL = frame[:, :int(self.width / 2), :]
                    frameR = frame[:, int(self.width / 2):, :]
                    yield idx, [frameL, frameR]
                else:
                    yield idx, [frame]
        finally:
            self.video_capture.release()
            self.video_capture = None
            
        raise StopIteration

            
def detect_chessboard(image, pattern_size, criteria, win_size, draw=False):
    # 检测棋盘格
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    better_corners = None
    found, corners = cv2.findChessboardCorners(gray, pattern_size)
    if found:
        # 角点精检测 (亚像素)
        better_corners = cv2.cornerSubPix(gray, corners, win_size, (-1, -1), criteria)
        if draw:
            cv2.drawChessboardCorners(image, pattern_size, better_corners, found)
    return better_corners


def calibrate_monocular_camera(ctx, imgfiles, world_point, pattern_size, criteria, win_size):        
    image_size = None
    world_points, pixel_points = [], []
    for ifile in imgfiles:
        image = cv2.imread(ifile)
        if image_size is None:
            image_size = image.shape[::-1][1:]
        pixel_point = detect_chessboard(image, pattern_size, criteria, win_size)
        if pixel_point is not None: 
            world_points.append(world_point)
            pixel_points.append(pixel_point)
 
    error, mtx, dist = start_calibrate_camera_with_good_flags(ctx,
        functools.partial(cv2.calibrateCameraExtended, objectPoints=world_points, imagePoints=pixel_points),
        img_size=image_size)
    
    return image_size, error, mtx, dist


def calibrate_binocular_camera(ctx, img_l_files, img_r_files, world_point, pattern_size, criteria, win_size):
    image_size = None
    world_points, pixel_points_L, pixel_points_R = [], [], []
    for lfile, rfile in zip(img_l_files, img_r_files):
        image_L = cv2.imread(lfile)
        image_R = cv2.imread(rfile)
        if image_size is None:
            image_size = image_L.shape[::-1][1:]
            ctx.logger(f'{lfile}, {rfile}, {image_size}')
        pixel_point_L = detect_chessboard(image_L, pattern_size, criteria, win_size)
        pixel_point_R = detect_chessboard(image_R, pattern_size, criteria, win_size)
        if pixel_point_L is not None and pixel_point_R is not None: 
            world_points.append(world_point)
            pixel_points_L.append(pixel_point_L)
            pixel_points_R.append(pixel_point_R)
 
    error_L, mtx_L, dist_L = start_calibrate_camera_with_good_flags(ctx,
        functools.partial(cv2.calibrateCameraExtended, objectPoints=world_points, imagePoints=pixel_points_L),
        img_size=image_size)
    
    error_R, mtx_R, dist_R = start_calibrate_camera_with_good_flags(ctx,
        functools.partial(cv2.calibrateCameraExtended, objectPoints=world_points, imagePoints=pixel_points_R),
        img_size=image_size)
    
    error, K1, D1, K2, D2, R, T, E, F = cv2.stereoCalibrate(
        world_points, pixel_points_L, pixel_points_R,
        mtx_L, dist_L, mtx_R, dist_R, image_size,
        R=None, # 第一和第二个摄像机之间的旋转矩阵
        T=None, # 第一和第二个摄像机之间的平移矩阵
        E=None, # essential matrix本质矩阵
        F=None, # fundamental matrix基本矩阵
        flags=None, criteria=None)
        # flags=cv2.CALIB_FIX_INTRINSIC, criteria=criteria)
    
    # D1 = np.zeros((5, 1))
    # D2 = np.zeros((5, 1))

    return image_size, error, K1, D1, K2, D2, R, T, E, F


def rectify_binocular_camera(ctx, K1, D1, K2, D2, image_size, R, T):
    R1, R2, P1, P2, Q, roi_left, roi_right = cv2.stereoRectify(
        K1, D1, K2, D2, image_size, R, T,
        R1=None, # 第一个摄像机的校正变换矩阵
        R2=None, # 第二个摄像机的校正变换矩阵
        P1=None, # 第一个摄像机在新坐标系下的投影矩阵
        P2=None, # 第二个摄像机在新坐标系下的投影矩阵
        Q=None,  # 4*4的视差图到深度图的映射矩阵
        alpha=0, newImageSize=None)
        # flags=cv2.CALIB_ZERO_DISPARITY, alpha=0.9, newImageSize=None)
    
    # 畸变校正和立体校正的映射矩阵, 计算像素空间坐标的重投影矩阵
    l_map_x, l_map_y = cv2.initUndistortRectifyMap(K1, D1, R1, P1, image_size, cv2.CV_32FC1)
    print(l_map_x)
    r_map_x, r_map_y = cv2.initUndistortRectifyMap(K2, D2, R2, P2, image_size, cv2.CV_32FC1)
    return l_map_x, l_map_y, r_map_x, r_map_y, R1, R2, P1, P2, Q

### Callback Events

In [80]:
def on_start_collect_samples(ctx, w_btn, camera_source, sample_size, flip, rm_out, save_dir, w_video):
    segs = os.path.basename(save_dir).split('_')
    squares_x, squares_y, square_size, win_size, term_iters, term_eps = *list(map(int, segs[:-1])), float(segs[-1])
    ctx.logger('on_start_collect_samples(%s, %d, %d, %d, %d, %d, %d, %f, %d)' % (
        ':'.join(camera_source), sample_size, squares_x, squares_y, square_size, win_size, term_iters, term_eps,
    flip), clear=1)
    
    pattern_size = (squares_x - 1, squares_y - 1)
    win_size = (win_size, win_size)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, term_iters, term_eps)
    
    if rm_out:
        shutil.rmtree(save_dir, ignore_errors=True)
    os.makedirs(save_dir, exist_ok=True)
    
    def _camera_capture(camera, w_video, save_dir):
        try:
            w_btn.disabled = True
            count = 0
            for idx, frames in camera.read(flip=flip):
                if idx % camera.fps == 0:
                    if len(frames) == 2:
                        frameL_copied, frameR_copied = frames[0].copy(), frames[1].copy()
                        cornersL = detect_chessboard(frameL_copied, pattern_size, criteria, win_size, draw=True)
                        cornersR = detect_chessboard(frameR_copied, pattern_size, criteria, win_size, draw=True)
                        if cornersL is not None and cornersR is not None:
                            count += 1
                            cv2.imwrite(os.path.join(save_dir, "left_orig_{:0=3d}.png".format(count)), frames[0])
                            cv2.imwrite(os.path.join(save_dir, "left_copy_{:0=3d}.png".format(count)), frameL_copied)

                            cv2.imwrite(os.path.join(save_dir, "right_orig_{:0=3d}.png".format(count)), frames[1])
                            cv2.imwrite(os.path.join(save_dir, "right_copy_{:0=3d}.png".format(count)), frameR_copied)

                            cv2.putText(frameL_copied, f'Count: {count}', (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, COLORS.BLUE, 2)
                            nbeasy_widget_display({'L': frameL_copied, 'R': frameR_copied}, w_video)

                            if count == sample_size:
                                break
                    else:
                        frame_copied = frames[0]
                        corners = detect_chessboard(frame_copied, pattern_size, criteria, win_size, draw=True)
                        if corners is not None:
                            count += 1
                            cv2.imwrite(os.path.join(save_dir, "orig_{:0=3d}.png".format(count)), frames[0])
                            cv2.imwrite(os.path.join(save_dir, "chess_{:0=3d}.png".format(count)), frame_copied)
                            cv2.putText(frame_copied, f'Count: {count}', (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, COLORS.BLUE, 2)
                            nbeasy_widget_display(frame_copied, w_video)

                            if count == sample_size:
                                break
        except (StopIteration, RuntimeError):
            pass
        finally:
            camera.close()
            w_btn.disabled = False

    camera = StereoCamera(camera_source[0], camera_source[1] == 'binocular')
    ctx.logger(f'video shape: {camera.open()}')
    ctx.stereo_camera = camera
    calibrate_thread = threading.Thread(target=_camera_capture, name='stereocamera', args=(camera, w_video, save_dir))
    calibrate_thread.daemon = True
    calibrate_thread.start()


def on_stop_collect_samples(ctx, w_btn = None):
    ctx.logger("on_stop_collect_samples")
    if hasattr(ctx, 'stereo_camera'):
        ctx.stereo_camera.close() 

        
def on_start_calibrition_test(ctx, w_btn, camera_source, flip, save_dir, calibration_result, w_video):
    ctx.logger('on_start_calibrition_test(%s, %s)' % (':'.join(camera_source), save_dir), clear=1)
    is_3d = camera_source[1] == 'binocular'
    if not is_3d:
        ctx.logger('not 3d camera')
        return
    
    def _camera_capture(camera, w_video, save_dir):
        try:
            w_btn.disabled = True
            params = json.loads(calibration_result)

            K1, D1 = np.asarray(params['K1']), np.asarray(params['D1'])
            K2, D2 = np.asarray(params['K2']), np.asarray(params['D2'])
            
            for idx, frames in camera.read(flip=flip):
                display_frames = {}
                if len(frames) == 2:
                    imgL, imgR = frames
                    display_frames['L'] = imgL
                    display_frames['R'] = imgR
                    display_frames['undist L'] = cv2.undistort(imgL.copy(), K1, D1)
                    display_frames['undist R'] = cv2.undistort(imgR.copy(), K2, D2)
                    nbeasy_widget_display(display_frames, w_video)
                else:
                    raise NotImplemented
        except (StopIteration, RuntimeError):
            pass
        finally:
            camera.close()
            w_btn.disabled = False

    camera = StereoCamera(camera_source[0], is_3d)
    ctx.logger(f'video shape: {camera.open()}')
    ctx.stereo_camera = camera
    calibrate_thread = threading.Thread(target=_camera_capture, name='stereocamera', args=(camera, w_video, save_dir))
    calibrate_thread.daemon = True
    calibrate_thread.start()
    
    
on_stop_test = on_stop_collect_samples


def on_start_rectification_test(ctx, w_btn, camera_source, flip, save_dir, rectification_result, w_video):
    ctx.logger('on_start_rectification_test(%s, %s)' % (':'.join(camera_source), save_dir), clear=1)
    is_3d = camera_source[1] == 'binocular'
    if not is_3d:
        ctx.logger('not 3d camera')
        return
    
    def _camera_capture(camera, w_video, save_dir):
        try:
            w_btn.disabled = True
            params = json.loads(rectification_result)

            K1, D1 = np.asarray(params['K1']), np.asarray(params['D1'])
            K2, D2 = np.asarray(params['K2']), np.asarray(params['D2'])
            
            l_map, r_map = np.load(params['l_map_npz']), np.load(params['r_map_npz'])
            l_map_x, l_map_y = l_map['l_map_x'], l_map['l_map_y']
            r_map_x, r_map_y = r_map['r_map_x'], r_map['r_map_y']
            
            for idx, frames in camera.read(flip=flip):
                display_frames = {}
                if len(frames) == 2:
                    imgL, imgR = frames
                    display_frames['undist L'] = cv2.undistort(imgL.copy(), K1, D1)
                    display_frames['undist R'] = cv2.undistort(imgR.copy(), K2, D2)
                    display_frames['rectified L'] = cv2.remap(imgL, l_map_x, l_map_y, cv2.INTER_LINEAR, borderValue=cv2.BORDER_CONSTANT)
                    display_frames['rectified R'] = cv2.remap(imgR, r_map_x, r_map_y, cv2.INTER_LINEAR, borderValue=cv2.BORDER_CONSTANT)
                    nbeasy_widget_display(display_frames, w_video)
                else:
                    raise NotImplemented
        except (StopIteration, RuntimeError):
            pass
        finally:
            camera.close()
            w_btn.disabled = False

    camera = StereoCamera(camera_source[0], is_3d)
    ctx.logger(f'video shape: {camera.open()}')
    ctx.stereo_camera = camera
    calibrate_thread = threading.Thread(target=_camera_capture, name='stereocamera', args=(camera, w_video, save_dir))
    calibrate_thread.daemon = True
    calibrate_thread.start()
    
    
def on_start_matcher(ctx, w_btn, camera_source, flip, save_dir, rectification_result, w_video):
    ctx.logger('on_start_matcher(%s, %s)' % (':'.join(camera_source), save_dir), clear=1)
    is_3d = camera_source[1] == 'binocular'
    if not is_3d:
        ctx.logger('not 3d camera')
        return
    
    def _camera_capture(camera, w_video, save_dir):
        try:
            w_btn.disabled = True
            params = json.loads(rectification_result)
            l_map, r_map = np.load(params['l_map_npz']), np.load(params['r_map_npz'])
            l_map_x, l_map_y = l_map['l_map_x'], l_map['l_map_y']
            r_map_x, r_map_y = r_map['r_map_x'], r_map['r_map_y']
            
            Q = np.asarray(params['Q'])
            
            matcherL = cv2.StereoSGBM_create(
                minDisparity=0,
                numDisparities=128,
                blockSize=3,
                P1=8 * 1 * 3 ** 2,
                P2=32 * 1 * 3 ** 2,
                disp12MaxDiff=1,
                preFilterCap=64,
                uniquenessRatio=15,
                speckleWindowSize=100,
                speckleRange=1,
                mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY)
            matcherR = cv2.ximgproc.createRightMatcher(matcherL)
            
            lmbda = 80000
            sigma = 1.3
            filter = cv2.ximgproc.createDisparityWLSFilter(matcher_left=matcherL)
            filter.setLambda(lmbda)
            filter.setSigmaColor(sigma)
            
            for idx, frames in camera.read(flip=flip):
                display_images = {}
                if len(frames) == 2:
                    imgL, imgR = frames
                    rectifiedL = cv2.remap(imgL, l_map_x, l_map_y, cv2.INTER_LINEAR, borderValue=cv2.BORDER_CONSTANT)
                    rectifiedR = cv2.remap(imgR, r_map_x, r_map_y, cv2.INTER_LINEAR, borderValue=cv2.BORDER_CONSTANT)
                    display_images['rectifiedL'] = rectifiedL
                    display_images['rectifiedR'] = rectifiedR
                    H = max(rectifiedL.shape[0], rectifiedR.shape[0])
                    W = rectifiedL.shape[1] + rectifiedR.shape[1]
                    rectify_image = np.zeros((H, W, 3), dtype=np.uint8)
                    rectify_image[0:rectifiedL.shape[0], 0:rectifiedL.shape[1]] = rectifiedL
                    rectify_image[0:rectifiedR.shape[0],  rectifiedL.shape[1]:] = rectifiedR
                    
                    interval = 50
                    # 绘制等间距平行线
                    for k in range(H // interval):
                        cv2.line(rectify_image, (0, interval * (k + 1)), (2 * W, interval * (k + 1)), COLORS.RED, 2, lineType=cv2.LINE_AA)
                        
                    grayL = cv2.cvtColor(rectifiedL, cv2.COLOR_BGR2GRAY)
                    grayR = cv2.cvtColor(rectifiedR, cv2.COLOR_BGR2GRAY)

                    dispL = np.uint16(matcherL.compute(grayL, grayR))
                    dispR = np.uint16(matcherR.compute(grayR, grayL))

                    dispL = np.uint16(filter.filter(dispL, grayL, None, dispR))
                    dispL[dispL < 0] = 0
                    dispL = dispL.astype(np.float32) / 16.
                    
                    points_3d = cv2.reprojectImageTo3D(dispL, Q)
                    points_3d = np.asarray(points_3d, dtype=np.float32)
                    
                    nbeasy_widget_display(rectify_image, w_video)
                else:
                    raise NotImplemented
        except (StopIteration, RuntimeError):
            pass
        finally:
            camera.close()
            w_btn.disabled = False

    camera = StereoCamera(camera_source[0], is_3d)
    ctx.logger(f'video shape: {camera.open()}')
    ctx.stereo_camera = camera
    calibrate_thread = threading.Thread(target=_camera_capture, name='stereocamera', args=(camera, w_video, save_dir))
    calibrate_thread.daemon = True
    calibrate_thread.start()

    
on_stop_matcher = on_stop_collect_samples


def on_previous_sample(ctx, w_btn, w_sample_list):
    if w_sample_list.index > 0:
        w_sample_list.index -= 1


def on_next_sample(ctx, w_btn, w_sample_list):
    if w_sample_list.index < len(w_sample_list.options):
        w_sample_list.index += 1


def on_remove_sample(ctx, w_btn, w_sample_list):
    sample_path = w_sample_list.value
    ctx.logger(f'on_remove_sample({sample_path})')
    if os.path.exists(sample_path):
        os.remove(sample_path)
        if 'binocular' in sample_path:
            os.remove(sample_path.replace('left', 'right'))
    options = [(_key, _val) for _key, _val in w_sample_list.options if _val != sample_path]
    w_sample_list.options = options


def on_refresh_sample(ctx, w_btn, w_sample_list, save_dir):
    ctx.logger(f'on_refresh_sample({save_dir})')
    
    options = []
    if 'binocular' in save_dir:
        left_sample_list = sorted(glob.glob(f'{save_dir}/left_chess_*.png'))
        right_sample_list = sorted(glob.glob(f'{save_dir}/right_chess_*.png'))
        ctx.logger(f'left samples count: {len(left_sample_list)}, right samples count: {len(right_sample_list)}')
        assert len(left_sample_list) == len(right_sample_list)
        for imgpath in left_sample_list:
            imgfile = os.path.basename(imgpath)
            options.append((imgfile[5:-4], imgpath))
    else:
        sample_list = sorted(glob.glob(f'{save_dir}/chess_*.png'))
        for imgpath in sample_list:
            imgfile = os.path.basename(imgpath)
            options.append((imgfile[:-4], imgpath))
    
    w_sample_list.options = options


def on_update_sample_dir(
    ctx, w_save_dir, w_calibration_result, w_rectification_result,
    camera_source, squares_x, squares_y, square_size, win_size, term_iters, term_eps):
    save_dir = f'out/{camera_source[1]}/chessboard/{squares_x}_{squares_y}_{square_size}_{win_size}_{term_iters}_{term_eps}'
    w_save_dir.value = save_dir
    ctx.logger(f'on_update_sample_dir({save_dir})')
    
    on_load_calibration_result(ctx, None, w_calibration_result, save_dir)
    if camera_source[1] == 'binocular':
        on_load_rectification_result(ctx, None, w_rectification_result, save_dir)


def on_update_sample_list(ctx, w_sample_list, save_dir):
    ctx.logger(f'on_update_sample_list({save_dir})')
    if not os.path.isdir(save_dir):
        return
    on_refresh_sample(ctx, None, w_sample_list, save_dir)


def on_update_sample_image(ctx, w_video_frame, sample_path):
    ctx.logger(f'on_update_sample_image({sample_path})')
    if sample_path is None or not os.path.exists(sample_path):
        ctx.logger(f'not found {sample_path}')
        return
    if 'binocular' in sample_path:
        imgL = cv2.imread(sample_path)
        imgR = cv2.imread(sample_path.replace('left', 'right'))
        nbeasy_widget_display({'L': imgL, 'R': imgR}, w_video_frame)
    else:
        img = cv2.imread(sample_path)
        nbeasy_widget_display(img, w_video_frame)


def on_start_calibration(ctx, w_btn, w_sample_list, w_calibration_result):
    try:
        w_btn.disabled = True
        w_calibration_result.value = ''
        options = w_sample_list.options
        ctx.logger(f'on_start_calibration({len(options)})')
        if len(options) == 0:
            return

        save_dir = os.path.dirname(options[0][1])
        segs = os.path.basename(save_dir).split('_')
        squares_x, squares_y, square_size, win_size, term_iters, term_eps = *list(map(int, segs[:-1])), float(segs[-1])

        pattern_size = (squares_x - 1, squares_y - 1)
        world_point = np.zeros((np.prod(pattern_size), 3), np.float32)
        world_point[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)
        world_point = world_point * square_size

        win_size = (win_size, win_size)
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, term_iters, term_eps)

        image_files = [x[1].replace('copy', 'orig') for x in options]
        if 'binocular' in save_dir:
            image_L_files = image_files
            image_R_files = [x.replace('left', 'right') for x in image_L_files]
            image_size, error, K1, D1, K2, D2, R, T, E, F = calibrate_binocular_camera(
                ctx, image_L_files, image_R_files, world_point, pattern_size, criteria, win_size)
            result = {
                'image_size': image_size,
                'error': error,
                'K1': K1, 'D1': D1,
                'K2': K2, 'D2': D2,
                'R': R, 'T': T, 'E': E, 'F': F
            }
        else:
            image_size, error, mtx, dist = calibrate_monocular_camera(ctx, image_files, world_point, pattern_size, criteria, win_size)
            result = {
                'image_size': image_size,
                'error': error,
                'K': mtx, 'D': dist
            }
        w_calibration_result.value = json.dumps(result, indent=4)
    finally:
        w_btn.disabled = False
        

def on_save_calibration_result(ctx, w_btn, calibration_result, save_dir):
    ctx.logger(f'on_save_calibration_result({save_dir})')
    prefix = 'binocular' if 'binocular' in save_dir else 'monocular'
    with open(f'{save_dir}/{prefix}_calibration.json', 'w') as f:
        f.write(calibration_result)
        
    
def on_load_calibration_result(ctx, w_btn, w_calibration_result, save_dir):
    ctx.logger(f'on_load_calibration_result({save_dir})')
    prefix = 'binocular' if 'binocular' in save_dir else 'monocular'
    try:
        file = f'{save_dir}/{prefix}_calibration.json'
        with open(file, 'r') as f:
            w_calibration_result.value = f.read()
    except Exception:
        w_calibration_result.value = f'Open {file} Error'
        
        
def on_start_rectify(ctx, w_btn, calibration_result, w_rectify_result, save_dir):
    ctx.logger('on_start_rectify')
    try:
        w_btn.disabled = True 
        w_rectify_result.value = ''
        params = json.loads(calibration_result)
        K1, D1 = np.asarray(params['K1']), np.asarray(params['D1'])
        K2, D2 = np.asarray(params['K2']), np.asarray(params['D2'])
        image_size, R, T = params['image_size'], np.asarray(params['R']), np.asarray(params['T'])
        l_map_x, l_map_y, r_map_x, r_map_y, R1, R2, P1, P2, Q = rectify_binocular_camera(
            ctx, K1, D1, K2, D2, image_size, R, T)
        
        np.savez(f'{save_dir}/l_map.npz', l_map_x=l_map_x, l_map_y=l_map_y)
        np.savez(f'{save_dir}/r_map.npz', r_map_x=r_map_x, r_map_y=r_map_y)
        
        result = {
            **params,
            'l_map_npz': f'{save_dir}/l_map.npz',
            'r_map_npz': f'{save_dir}/r_map.npz',
            'R1': R1, 'R2': R2, 'P1': P1, 'P2': P2, 'Q': Q
        }
        w_rectify_result.value = json.dumps(result, indent=4)
    except Exception:
        ctx.logger(f'{traceback.format_exc(limit=6)}')
    finally:
        w_btn.disabled = False
    

def on_save_rectification_result(ctx, w_btn, rectification_result, save_dir):
    ctx.logger(f'on_save_rectification_result({save_dir})')
    prefix = 'binocular' if 'binocular' in save_dir else 'monocular'
    with open(f'{save_dir}/{prefix}_rectification.json', 'w') as f:
        f.write(rectification_result)
        
    
def on_load_rectification_result(ctx, w_btn, w_rectification_result, save_dir):
    ctx.logger(f'on_load_rectification_result({save_dir})')
    prefix = 'binocular' if 'binocular' in save_dir else 'monocular'
    try:
        file = f'{save_dir}/{prefix}_rectification.json'
        with open(file, 'r') as f:
            w_rectification_result.value = f.read()
    except Exception:
        w_rectification_result.value = f'Open {file} Error'

### Template Schema

In [81]:
# _IMPORT('./easy_widget.py')

schema = {
    'type': 'page',
    'objs': [
        {
            'type': 'tab',
            'objs': [
                {
                    'name': 'Capture',
                    'objs': [
                        {
                            'type': 'V',
                            'name': 'Chessboard',
                            'objs': [
                                {
                                    'type': 'H',
                                    'objs': [
                                        nbeasy_widget_int('cfg.chessboard_squares_x', 'Squares X', 9),
                                        nbeasy_widget_int('cfg.chessboard_squares_y', 'Squares Y', 7),
                                        nbeasy_widget_int('cfg.chessboard_square_size', 'Square Size(mm)', 30)        
                                    ],
                                },
                                {
                                    'type': 'H',
                                    'objs': [
                                        nbeasy_widget_int('cfg.chessboard_win_size', 'Win Size', default=5),
                                        nbeasy_widget_int('cfg.chessboard_term_iters', 'Term Iters', default=500),
                                        nbeasy_widget_float('cfg.chessboard_term_eps', 'Term EPS', default=0.0001),
                                    ]
                                },
                            ]
                        }, # end Chessboard
                        { 'type': 'html', 'text': '<hr>'},
                        {
                            'type': 'V',
                            'name': 'Samples',
                            'objs': [
                                {
                                    'type': 'H',
                                    'objs': [
                                        nbeasy_widget_stringenum('cfg.select_camera_source', 'Select Camera', enums=_get_cameras()),
                                        nbeasy_widget_int('cfg.sample_size', 'Sample Size', 32),
                                        nbeasy_widget_stringenum('cfg.select_flip', 'Flip Type', enums=[
                                            ('None', -2),
                                            ('Horizontal', 1),
                                            ('Vertical', 0),
                                            ('Both', -1)]),
                                    ]
                                },
                                {
                                    'type': 'H',
                                    'objs': [
                                        nbeasy_widget_string('cfg.sample_save_dir', 'Save Dir', "", width=605, readonly=True),
                                        nbeasy_widget_bool('cfg.rm_out', '<font color="red">Clear Images</font>', False),
                                    ]
                                },
                                {
                                    'type': 'V',
                                    'objs': [
                                        {
                                            'type': 'H',
                                            'objs': [
                                                nbeasy_widget_button('__cfg.btn_start_collect_samples', 'Start', style='success', icon='camera'),
                                                nbeasy_widget_button('__cfg.btn_stop_collect_samples', 'Stop', style='success', icon='stop-circle'),
                                            ],
                                            'justify_content': 'center'
                                        },
                                        nbeasy_widget_image('__cfg.video_frame_capture', 'Frame', '', width=640, height=480),
                                        {
                                            'type': 'H',
                                            'objs': [
                                                nbeasy_widget_button('__cfg.btn_previous_sample', 'Previous', style='info', icon='arrow-left', width=100),
                                                nbeasy_widget_button('__cfg.btn_next_sample', 'Next', style='info', icon='arrow-right', width=100),
                                                nbeasy_widget_stringenum('cfg.sample_list', '', default=0, enums=['NONE'], width=200, description_width=0),
                                                nbeasy_widget_button('__cfg.btn_del_sample', 'Remove', style='danger', icon='trash', width=100),
                                                nbeasy_widget_button('__cfg.btn_refresh_sample', 'Refresh', style='info', icon='refresh', width=100),
                                            ],
                                            'justify_content': 'center'
                                        }
                                    ],
                                    'align_items': 'center'
                                },
                            ],
                        }, # end Samples
                    ]
                }, # end tab Capture
                {
                    'name': 'Calibrate',
                    'objs': [
                        {
                            'type': 'H',
                            'objs': [
                                {
                                    'type': 'V',
                                    'objs': [
                                        {
                                            'type': 'H',
                                            'objs': [
                                                nbeasy_widget_button('__cfg.start_calibration', 'Calibrate', style='success', icon='check'),
                                                nbeasy_widget_button('__cfg.save_calibration_result', 'Save', style='success', icon='floppy-o'),
                                                nbeasy_widget_button('__cfg.load_calibration_result', 'Load', style='success', icon='spinner'),
                                            ],
                                            'justify_content': 'center'
                                        }, 
                                        {
                                            'type': 'H',
                                            'objs': [
                                                nbeasy_widget_text('cfg.calibration_result', '', width=500, height=300)
                                            ],
                                            'justify_content': 'center'
                                        },
                                    ],
                                    'width': '50%'
                                },
                                {
                                    'type': 'V',
                                    'objs': [
                                        {
                                            'type': 'H',
                                            'objs': [
                                                nbeasy_widget_button('__cfg.start_rectification', 'Rectify', style='success', icon='check'),
                                                nbeasy_widget_button('__cfg.save_rectification_result', 'Save', style='success', icon='floppy-o'),
                                                nbeasy_widget_button('__cfg.load_rectification_result', 'Load', style='success', icon='spinner'),
                                            ],
                                            'justify_content': 'center'
                                        },
                                        {
                                            'type': 'H',
                                            'objs': [
                                                nbeasy_widget_text('cfg.rectification_result', '', width=520, height=300)
                                            ],
                                            'justify_content': 'center'
                                        },
                                    ],
                                    'width': '50%'
                                },
                            ],
                        }, # end H
                        {
                            'type': 'V',
                            'objs': [
                                {
                                    'type': 'H',
                                    'objs': [
                                        nbeasy_widget_button('__cfg.btn_start_c_test',  'Test Calibration', style='success', icon='camera'),
                                        nbeasy_widget_button('__cfg.btn_stop_test', 'Stop Test', style='success', icon='stop-circle'),
                                        nbeasy_widget_button('__cfg.btn_start_r_test', 'Test Rectification', style='success', icon='camera'),
                                    ],
                                    'justify_content': 'center'
                                },
                                nbeasy_widget_image('__cfg.video_frame_test', 'Frame', '', width=640, height=480),
                            ],
                            'align_items': 'center'
                        },
                    ]
                }, # end tab Calibration
                {
                    'name': 'Matcher',
                    'objs': [
                        {
                            'type': 'V',
                            'objs': [
                                {
                                    'type': 'V',
                                    'objs': [
                                        {
                                            'type': 'H',
                                            'objs': [
                                                nbeasy_widget_button('__cfg.btn_start_matcher', 'Start', style='success', icon='camera'),
                                                nbeasy_widget_button('__cfg.btn_stop_matcher', 'Stop', style='success', icon='stop-circle'),
                                            ],
                                            'justify_content': 'center'
                                        },
                                        nbeasy_widget_image('__cfg.video_frame_matcher', 'Frame', '', width=640, height=480),
                                    ],
                                    'align_items': 'center'
                                },
                            ],
                        },
                    ]
                }, # end tab Matcher
            ]
        }, # end tab
    ], # end pages
    'evts': [
        {
            'type': 'jsdlink',
            'objs': [
                {
                    'source': '__cfg.btn_start_collect_samples:disabled',
                    'target': '__cfg.btn_start_matcher:disabled'
                },
                {
                    'source': '__cfg.btn_start_collect_samples:disabled',
                    'target': '__cfg.btn_start_c_test:disabled'
                },
                {
                    'source': '__cfg.btn_start_collect_samples:disabled',
                    'target': '__cfg.btn_start_r_test:disabled'
                }
            ]
        }, # end jsdlink
        {
            'type': 'onclick',
            'objs': [
                {
                    'handler': on_start_collect_samples,
                    'params': {
                        'sources': ['__cfg.btn_start_collect_samples'],
                        'targets': [
                            'cfg.select_camera_source:value',
                            'cfg.sample_size:value',
                            'cfg.select_flip:value',
                            'cfg.rm_out:value',
                            'cfg.sample_save_dir:value',
                            '__cfg.video_frame_capture',
                        ]
                    }
                }, # end event start collect samples
                {
                    'handler': on_stop_collect_samples,
                    'params': {
                        'sources': ['__cfg.btn_stop_collect_samples'],
                        'targets': []
                    }
                }, # end event stop collect samplesn
                {
                    'handler': on_start_calibrition_test,
                    'params': {
                        'sources': ['__cfg.btn_start_c_test'],
                        'targets': [
                            'cfg.select_camera_source:value',
                            'cfg.select_flip:value',
                            'cfg.sample_save_dir:value',
                            'cfg.calibration_result:value',
                            '__cfg.video_frame_test',
                        ]
                    }
                }, # end event start c test
                {
                    'handler': on_stop_test,
                    'params': {
                        'sources': ['__cfg.btn_stop_test'],
                        'targets': []
                    }
                }, # end event stop c test
                {
                    'handler': on_start_rectification_test,
                    'params': {
                        'sources': ['__cfg.btn_start_r_test'],
                        'targets': [
                            'cfg.select_camera_source:value',
                            'cfg.select_flip:value',
                            'cfg.sample_save_dir:value',
                            'cfg.rectification_result:value',
                            '__cfg.video_frame_test',
                        ]
                    }
                }, # end event start c test
                {
                    'handler': on_start_matcher,
                    'params': {
                        'sources': ['__cfg.btn_start_matcher'],
                        'targets': [
                            'cfg.select_camera_source:value',
                            'cfg.select_flip:value',
                            'cfg.sample_save_dir:value',
                            'cfg.rectification_result:value',
                            '__cfg.video_frame_matcher',
                        ]
                    }
                }, # end event start matcher
                {
                    'handler': on_stop_matcher,
                    'params': {
                        'sources': ['__cfg.btn_stop_matcher'],
                        'targets': []
                    }
                }, # end event stop matcher
                {
                    'handler': on_previous_sample,
                    'params': {
                        'sources': ['__cfg.btn_previous_sample'],
                        'targets': [
                            'cfg.sample_list'
                        ]
                    }
                }, # end event previous sample
                {
                    'handler': on_next_sample,
                    'params': {
                        'sources': ['__cfg.btn_next_sample'],
                        'targets': [
                            'cfg.sample_list'
                        ]
                    }
                }, # end even next sample
                {
                    'handler': on_remove_sample,
                    'params': {
                        'sources': ['__cfg.btn_del_sample'],
                        'targets': [
                            'cfg.sample_list' 
                        ]
                    }
                }, # end event remove sample
                {
                    'handler': on_refresh_sample,
                    'params': {
                        'sources': ['__cfg.btn_refresh_sample'],
                        'targets': [
                            'cfg.sample_list',
                            'cfg.sample_save_dir:value'
                        ] 
                    }
                }, # end event refresh sample
                {
                    'handler': on_start_calibration,
                    'params': {
                        'sources': ['__cfg.start_calibration'],
                        'targets': [
                            'cfg.sample_list',
                            'cfg.calibration_result',
                        ]
                    }
                }, # end event calibrator
                {
                    'handler': on_save_calibration_result,
                    'params': {
                        'sources': ['__cfg.save_calibration_result'],
                        'targets': [
                            'cfg.calibration_result:value',
                            'cfg.sample_save_dir:value'
                        ]
                    }
                }, # end event on save calibration result
                {
                    'handler': on_load_calibration_result,
                    'params': {
                        'sources': ['__cfg.load_calibration_result'],
                        'targets': [
                            'cfg.calibration_result',
                            'cfg.sample_save_dir:value'
                        ]
                    }
                }, # end event on load calibration result
                {
                    'handler': on_start_rectify,
                    'params': {
                        'sources': ['__cfg.start_rectification'],
                        'targets': [
                            'cfg.calibration_result:value',
                            'cfg.rectification_result',
                            'cfg.sample_save_dir:value'
                        ]
                    }
                }, # end event rectify
                {
                    'handler': on_save_rectification_result,
                    'params': {
                        'sources': ['__cfg.save_rectification_result'],
                        'targets': [
                            'cfg.rectification_result:value',
                            'cfg.sample_save_dir:value'
                        ]
                    }
                }, # end event on save rectification result
                {
                    'handler': on_load_rectification_result,
                    'params': {
                        'sources': ['__cfg.load_rectification_result'],
                        'targets': [
                            'cfg.rectification_result',
                            'cfg.sample_save_dir:value'
                        ]
                    }
                }, # end event on load rectification result
            ]
        }, # end onclick events
        {
            'type': 'interactiveX',
            'objs': [
                {
                    'handler': on_update_sample_dir,
                    'params': {
                        'w_save_dir': 'cfg.sample_save_dir',
                        'w_calibration_result': 'cfg.calibration_result',
                        'w_rectification_result': 'cfg.rectification_result',
                        'camera_source': 'cfg.select_camera_source',
                        'squares_x': 'cfg.chessboard_squares_x',
                        'squares_y': 'cfg.chessboard_squares_y',
                        'square_size': 'cfg.chessboard_square_size',
                        'win_size': 'cfg.chessboard_win_size',
                        'term_iters': 'cfg.chessboard_term_iters',
                        'term_eps': 'cfg.chessboard_term_eps',
                    }
                },
                {
                    'handler': on_update_sample_list,
                    'params': {
                        'w_sample_list': 'cfg.sample_list',
                        'save_dir': 'cfg.sample_save_dir',
                    }
                }, 
                {
                    'handler': on_update_sample_image,
                    'params': {
                        'w_video_frame': '__cfg.video_frame_capture',
                        'sample_path': 'cfg.sample_list'
                    }
                }
            ]
        } # end interactiveX
    ] # end events
}

if g_ctx:
    on_stop_collect_samples(g_ctx)
g_ctx = nbeasy_schema_parse(schema, debug=True, border=False)

Box(children=(Box(children=(VBox(children=(Tab(children=(VBox(children=(HTML(value="<b><font color='black'>Che…

In [15]:
raise

RuntimeError: No active exception to reraise

## UnD-1

In [ ]:
def nbon_start_camera(ctx, sbtn,
                      w_bgsub_method, w_bgsub_history, w_det_shadow, 
                      w_rmn_shadow_threshold, w_rmn_morpho_kernel, rmn_erode_iter, rmn_dilate_iter,
                      w_cnt_area_threshold,
                      w_cam_frame):
    method = str.lower(w_bgsub_method.value)
    history_length = w_bgsub_history.value
    detect_shadows = w_det_shadow.value
    
    shadow_thresh = w_rmn_shadow_threshold.value
    kernel = np.ones((w_rmn_morpho_kernel.value, w_rmn_morpho_kernel.value), np.uint8)
    
    erode_iter = rmn_erode_iter.value
    dilate_iter = rmn_dilate_iter.value
    
    area_threshold = w_cnt_area_threshold.value
    
    ctx.logger(f'nbon_start_camera({method}, {history_length}, {detect_shadows}, {shadow_thresh}, {w_rmn_morpho_kernel.value})')
    
    import threading
    
    def _video_capture():
        sbtn.disabled = True
        parameters = cv2.aruco.DetectorParameters_create()
        aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_5X5_50)
        pixel_cm_ratio_1, pixel_cm_ratio_2 = -1, -1
        marker_corner_1, marker_corner_2 = None, None
        try:
            bg_object = None
            if method == 'knn':
                bg_object = cv2.createBackgroundSubtractorKNN(history=history_length, detectShadows=detect_shadows)
            elif method == 'mog2':
                bg_object = cv2.createBackgroundSubtractorMOG2(history=history_length, detectShadows=detect_shadows)

            camera = cv2.VideoCapture(0)
            width = int(camera.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT))

            ctx.camera = camera
            ctx.camera_is_running = ctx.camera.isOpened()
            while ctx.camera_is_running:
                ret, frame_bgr = camera.read()
                if not ret:
                    break
                    
                # Detect Aruco markers 5cm x 5cm
                corners, ids, _ = cv2.aruco.detectMarkers(frame_bgr, aruco_dict, parameters=parameters)
                if corners is None or len(corners) == 0: 
                    w_cam_frame.value = io.BytesIO(cv2.imencode('.png', frame_bgr)[1]).getvalue()
                    continue
                    
                frame_copy = frame_bgr.copy()
                    
                for (marker_corner, marker_ID) in zip(corners, ids):
                    (tl, tr, br, bl) = [(int(x[0]), int(x[1])) for x in marker_corner.reshape((4, 2))]
                    rect = cv2.minAreaRect(marker_corner[0])
                    if marker_ID == 1:
                        pixel_cm_ratio_1 = cv2.arcLength(marker_corner, True) / (4 * 5) # place it on the target left
                        marker_corner_1 = (tl, tr, br, bl)
                        cv2.putText(frame_copy,
                                    'id:1 {}x{}'.format(round(rect[1][0], 2), round(rect[1][1], 2)),
                                    (marker_corner_1[0][0], marker_corner_1[0][1] - 15),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
                    elif marker_ID == 2:
                        pixel_cm_ratio_2 = cv2.arcLength(marker_corner, True) / (4 * 5) # place it on the target right
                        marker_corner_2 = (tl, tr, br, bl)
                        cv2.putText(frame_copy,
                                    'id:2 {}x{}'.format(round(rect[1][0], 2), round(rect[1][1], 2)),
                                    (marker_corner_2[0][0], marker_corner_2[0][1] - 15),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
                        
                if marker_corner_1 is None and marker_corner_2 is None:
                    w_cam_frame.value = io.BytesIO(cv2.imencode('.png', frame_bgr)[1]).getvalue()
                    continue
                                                             
                cv2.aruco.drawDetectedMarkers(frame_copy, corners, ids, borderColor=(255, 0, 0))

                if bg_object:
                    # Apply the background object on the frame to get the segmented mask.     
                    fgmask = bg_object.apply(frame_bgr)

                    # Perform thresholding to get rid of the shadows.
                    _, fgmask = cv2.threshold(fgmask, shadow_thresh, 255, cv2.THRESH_BINARY)

                    # Apply some morphological operations to make sure you have a good mask
                    fgmask = cv2.erode(fgmask, kernel, iterations=erode_iter)
                    fgmask = cv2.dilate(fgmask, kernel, iterations=dilate_iter)

                    # Get foreground object
                    foreground = cv2.bitwise_and(frame_bgr, frame_bgr, mask=fgmask)
                    
                    frame_mid = foreground
                    frame_dst = fgmask
                else:
                    # Grayscale & Guassian blur
                    frame_gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY) 
                    frame_blur = cv2.GaussianBlur(frame_gray, (3, 3), 0)

                    # # Divide gray by morphology image
                    # frame_div = cv2.divide(frame_gray, frame_blur, scale=255)

                    # Sharpen using unsharp masking
                    # frame_sharp = filters.unsharp_mask(frame_div, radius=1.5, amount=1.5, channel_axis=None, preserve_range=False)
                    # frame_sharp = (255 * frame_sharp).clip(0, 255).astype(np.uint8)

                    # Otsu Filter
                    # _, frame_otsu = cv2.threshold(frame_blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
                    _, frame_otsu = cv2.threshold(frame_gray, 0, 255, cv2.THRESH_OTSU)

                    # Detect edge by candy
                    # frame_canny = cv2.Canny(frame_blur, 200, 250)
                    
                    frame_mid = cv2.cvtColor(frame_otsu, cv2.COLOR_GRAY2BGR)
                    frame_dst = frame_otsu

                contours, _ = cv2.findContours(frame_dst, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
                for cnt in contours:
                    if cv2.contourArea(cnt) > area_threshold:
                        x1, y1, width, height = cv2.boundingRect(cnt)
                        x2, y2 = x1 + width, y1 + height
                        
                        cv2.rectangle(frame_copy, (x1, y1), (x2, y2),(0, 0, 255), 2)
                        cv2.putText(frame_copy,
                                    '{}x{}'.format(round(width, 2), round(height, 2)),
                                    (x2, y2),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
                        
                        if marker_corner_1:
                            if x1 < marker_corner_1[1][0] or x1 < marker_corner_1[2][0]:
                                continue
                        if marker_corner_2:
                            if x2 > marker_corner_2[0][0] or x2 > marker_corner_2[3][0]:
                                continue
                        
                        rect = cv2.minAreaRect(cnt)
                        (cx, cy), (cw, ch), angle = rect
                        
                        if marker_corner_1 and marker_corner_2:
                            ow = round(0.5 * cw / pixel_cm_ratio_1 + 0.5 * cw / pixel_cm_ratio_2, 2)
                            oh = round(0.5 * ch / pixel_cm_ratio_1 + 0.5 * ch / pixel_cm_ratio_2, 2)
                        elif marker_corner_1:
                            ow = round(cw / pixel_cm_ratio_1, 2)
                            oh = round(ch / pixel_cm_ratio_1, 2)
                        else:
                            ow = round(cw / pixel_cm_ratio_2, 2)
                            oh = round(ch / pixel_cm_ratio_2, 2)
                        
                        cv2.circle(frame_copy, (int(cx), int(cy)), 10, (255, 0, 0), -1)
                        cv2.polylines(frame_copy, [np.int0(cv2.boxPoints(rect))], True, (255, 0, 0), 2)
                        cv2.putText(frame_copy, "{}x{}".format(ow, oh), (int(cx - 100), int(cy - 20)), cv2.FONT_HERSHEY_PLAIN, 0.5, (100, 200, 0), 1)

                stacked = np.hstack((frame_bgr, frame_mid, frame_copy))
                w_cam_frame.value = io.BytesIO(cv2.imencode('.png', stacked)[1]).getvalue()
                
        except Exception as err:
            ctx.logger(f'{err}')
        finally:
            sbtn.disabled = False
            ctx.camera.release()
            ctx.camera_is_running = False
            ctx.camera = None
        
    
    camera_thread = threading.Thread(target=_video_capture, name='camera')
    camera_thread.daemon = True
    camera_thread.start()

    
def nbon_stop_camera(ctx, sbtn):
    ctx.logger('nbon_stop_camera()')
    if ctx.camera:
        ctx.camera_is_running = False
        for i in range(3):
            if ctx.camera is None:
                break
            time.sleep(1)
        if ctx.camera:
            ctx.camera.release()

In [ ]:
schema = {
    'type': 'page',
    'objs': [
        {
            'type': 'tab',
            'objs': [
                {
                    'name': 'Configuration',
                    'objs': [
                        {
                            'type': 'H',
                            'name': 'Background Subtraction',
                            'objs': [
                                nbeasy_widget_stringenum('cfg.bgsub_method', 'BG Sub Method', 1, enums=['None', 'MOG2', 'KNN'], width=300),
                                nbeasy_widget_int('cfg.bgsub_history', 'History Length', '300', min_=100, max_=600),
                                nbeasy_widget_bool('cfg.bgsub_det_shadow', 'Detect Shadows', True)
                            ]
                        },
                        {
                            'type': 'H',
                            'name': 'Remove Noise',
                            'objs': [
                                nbeasy_widget_int('cfg.rmn_shadow_threshold', 'Shadow Threshold', 250, min_=1, max_=255),
                                nbeasy_widget_stringenum('cfg.rmn_morpho_kernel', 'Morpho Kernel', 0, enums=[('3', 3), ('5', 5), ('7', 7), ('9', 9)], width=300),
                                nbeasy_widget_int('cfg.rmn_erode_iter', 'Erode Iters', 1),
                                nbeasy_widget_int('cfg.rmn_dilate_iter', 'Dilate Iters', 1),
                            ]
                        },
                        {
                            'type': 'H',
                            'name': 'Find Contours',
                            'objs': [
                                nbeasy_widget_float('cfg.cnt_area_threshold', 'Area Threshold', 2000),
                            ]
                        },
                        {
                            'type': 'V',
                            'name': 'Camera',
                            'objs': [
                                {
                                    'type': 'H',
                                    'objs': [
                                        nbeasy_widget_button('cfg.btn_start_camera', 'Start', width=200, style='success'),
                                        nbeasy_widget_button('cfg.btn_stop_camera', 'Stop', width=200, style='success')
                                    ],
                                    'justify_content': 'center'
                                },
                                nbeasy_widget_image('__cfg.camera_frame', 'Frame', '', height=480)
                            ],
                            'align_items': 'center'
                        }
                    ]
                }
            ]
        },
        {
            'type': 'onclick',
            'objs': [
                {
                    'handler': nbon_start_camera,
                    'params': {
                        'sources': ['cfg.btn_start_camera'],
                        'targets': [
                            'cfg.bgsub_method',
                            'cfg.bgsub_history',
                            'cfg.bgsub_det_shadow',
                            'cfg.rmn_shadow_threshold',
                            'cfg.rmn_morpho_kernel',
                            'cfg.rmn_erode_iter',
                            'cfg.rmn_dilate_iter',
                            'cfg.cnt_area_threshold',
                            '__cfg.camera_frame'
                        ]
                    }
                },
                {
                    'handler': nbon_stop_camera,
                    'params': {
                        'sources': ['cfg.btn_stop_camera'],
                        'targets': [
                        ]
                    }
                }
            ]
        }
    ]
}

if G:
    if hasattr(easy, 'camera') and easy.camera:
        easy.camera_is_running = False
        time.sleep(1)
        if easy.camera:
            easy.camera.release()
easy = nbeasy_schema_parse(schema, debug=True)
G = easy

In [ ]:
raise

In [ ]:
usb0.start()

## UnDef-2

In [ ]:
from IPython.display import display, Image

In [ ]:
parameters = cv2.aruco.DetectorParameters_create()
aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_5X5_50)

In [ ]:
display_handle = display(None, display_id=True)

try:
    cap = cv2.VideoCapture(0)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(width, height)
    ret, prev_frame= cap.read()
    while True:
        _, frame_bgr = cap.read()
        
        # frame_diff = cv2.absdiff(frame_bgr, prev_frame)
        # prev_frame = frame_bgr.copy()
        # gray = cv2.cvtColor(frame_diff, cv2.COLOR_BGR2GRAY)
        # blur = cv2.GaussianBlur(gray, (5, 5), 0)   
        # thresh = cv2.threshold(blur, 10, 255, cv2.THRESH_BINARY)[1]
        # dilate = cv2.dilate(thresh, None, iterations=5)
        # contours = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]

        # cv2.drawContours(frame_bgr, contours, -1, (0, 0, 0), 3)

        motion = 0
        # for cnt in contours:
        #     mask = np.zeros([height, width], dtype=np.uint8)
        #     area = cv2.contourArea(cnt)
        #     if area > 10000:
        #         motion += 1
        #         # rect = cv2.minAreaRect(cnt)
        #         # (x, y), (w, h), angle = rect
        #         # object_width = w / pixel_cm_ratio
        #         # object_height = h / pixel_cm_ratio
        #         x, y, w, h = cv2.boundingRect(cnt)
        #         x1 = x if x < 5 else x - 5
        #         y1 = y if y < 5 else y - 5
        #         x2 = x + w if (x + w + 5) > width else x + w + 5
        #         y2 = y + h if (y + h + 5) > height else y + h + 5
        #         # cv2.rectangle(frame_bgr, (x1, y1), (x2, y2), (255, 255, 255), -1)
        #         mask[y1:y2, x1:x2] = 255
        #         frame_mask = cv2.add(frame_bgr, np.zeros_like(frame_bgr, dtype=np.uint8), mask=mask)
                  
        if motion > -1:
            corners, ids, _ = cv2.aruco.detectMarkers(frame_bgr, aruco_dict, parameters=parameters)
            # cv2.aruco.drawDetectedMarkers(frame_bgr, corners, ids)
            if corners and len(corners) > 0: 
                aruco_perimeter = cv2.arcLength(corners[0], True)
                pixel_cm_ratio = aruco_perimeter / 20
                frame_gray = cv2.cvtColor(frame_bgr.copy(), cv2.COLOR_BGR2GRAY)
                img_blur = cv2.GaussianBlur(frame_gray, (5, 5), 0)
                img_edged = cv2.Canny(img_blur, 50, 100)
                img_edged = cv2.dilate(img_edged, None, iterations=1)
                img_edged = cv2.erode(img_edged, None, iterations=1)
                # frame_thresh = cv2.adaptiveThreshold(frame_gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 19, 5)
                contours = cv2.findContours(img_edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
                for cnt in contours:
                    area = cv2.contourArea(cnt)
                    # if area > 5000 and area < 153600:
                    if area > 1000 and area < 20000:
                        # cv2.drawContours(frame_bgr, [cnt], 0, color=(0, 0, 0), thickness=4)
                        rect = cv2.minAreaRect(cnt)
                        (x, y), (w, h), angle = rect
                        object_width = w / pixel_cm_ratio
                        object_height = h / pixel_cm_ratio
                        box = cv2.boxPoints(rect)
                        box = np.int0(box)
 
                        cv2.circle(frame_bgr, (int(x), int(y)), 10, (0, 0, 255), -1)
                        cv2.polylines(frame_bgr, [box], True, (100, 10, 0), 2)
                        # cv2.putText(frame_bgr, "{} cm".format(round(object_width, 2)), (int(x - 100), int(y - 20)), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 0), 2)
                        cv2.putText(frame_bgr, "{} cm".format(round(object_height, 2)), (int(x - 50), int(y + 15)), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 0), 2)
        
        _, frame = cv2.imencode('.jpeg', frame_bgr)
        display_handle.update(Image(data=frame.tobytes()))
except KeyboardInterrupt:
    pass
finally:
    cap.release()
    display_handle.update(None)

## References

- [旋转矩阵](https://slash-honeydew-c53.notion.site/a88e94293aeb4b54a729ceeb2f40a353)

1. Part 1: Image formation and pinhole model of the camera - https://towardsdatascience.com/image-formation-and-pinhole-model-of-the-camera-53872ee4ee92
2. Part 2: Camera Extrinsic Matrix in Python - https://towardsdatascience.com/camera-extrinsic-matrix-with-example-in-python-cfe80acab8dd
3. Part 3: Camera Intrinsic Matrix in Python - https://towardsdatascience.com/camera-intrinsic-matrix-with-example-in-python-d79bf2478c12
4. Part 4: Find the Minimum Stretching Direction of Positive Definite Matrices - https://towardsdatascience.com/find-the-minimum-stretching-direction-of-positive-definite-matrices-79c2a3b397fc
5. Part 5: Camera Calibration in Python - https://towardsdatascience.com/camera-calibration-with-example-in-python-5147e945cdeb
You can also find all the code in the GitHub repository - https://github.com/wingedrasengan927/Image-formation-and-camera-calibration


https://programtalk.com/vs4/python/OteRobotics/realant/pose_estimation.py/

https://its201.com/article/libo1004/110851205

https://string.quest/read/6034570